# Intro

This is the main notebook for the Capstone Project course

In [864]:
# Standard Libraries
import pandas as pd
import numpy as np
import os
import warnings
import json

# Plotting Libraries
import matplotlib.pyplot as plt
import seaborn as sns

#sklearn
from sklearn.preprocessing import OrdinalEncoder

# set Theme
plt.style.use('seaborn')
sns.set_theme(style="whitegrid")
warnings.filterwarnings('ignore')

In [865]:
# Set Paths
path = os.getcwd()
sourceDataPath = 'C:/Users/TheCu/OneDrive/Documents/Grad-School-Docs/CapstoneProject/SourceData/'
exportPath = path +'\\Export\\'
trimmedCSVPath = path +'\\SourceData\\'
cleanDataPath = path + '\\CleanData\\'

print(f"Source Data:",sourceDataPath)
print(f"Exports:",exportPath)
print(f"Trimmed CSV:",trimmedCSVPath)
print(f"Clean Data Export CSV:",cleanDataPath)

Source Data: C:/Users/TheCu/OneDrive/Documents/Grad-School-Docs/CapstoneProject/SourceData/
Exports: C:\Users\TheCu\OneDrive\Documents\Grad-School-Docs\CapstoneProject\Repo\Export\
Trimmed CSV: C:\Users\TheCu\OneDrive\Documents\Grad-School-Docs\CapstoneProject\Repo\SourceData\
Clean Data Export CSV: C:\Users\TheCu\OneDrive\Documents\Grad-School-Docs\CapstoneProject\Repo\CleanData\


# Resuable Functions

In [866]:
# Create Large CSV Trim Function

def largeCSVTrim(largeCSVPath, CSVName, trimQuery):
    tempDF = pd.read_csv(largeCSVPath+CSVName)
    df = pd.concat((x.query(trimQuery) for x in tempDF), ignore_index=True)
    return df

In [867]:
# Takes a DF and Producecs a Report of Null Values
def nullAnalysis(df):
    row = []
    null_summaryDF = pd.DataFrame(columns = ["Variable","Observations","Nulls","Null_Per","Num_Unique","Type"])
    for column in df:
        var = column
        countOfObs = len(df[var])
        countOfNull = df[var].isnull().sum()
        perOfNull = round((100 * countOfNull) / countOfObs,3)
        numUnique = df[var].nunique()
        type  = df[var].dtypes
        # Append to Summary DF
        row = [var, countOfObs,countOfNull,perOfNull,numUnique,type]
        df_length = len(null_summaryDF)
        null_summaryDF.loc[df_length] = row
    # Get Summary Stats
    countofVars = len(null_summaryDF)
    countofNotNull = (null_summaryDF['Nulls'] == 0).sum()
    countofNull = (null_summaryDF['Nulls'] != 0).sum()
    print(f"Count of Variables:",countofVars)
    print(f"Count of Variables without nulls:",countofNotNull)
    print(f"Count of Variables with null :",countofNull)
    # Format Summary
    summaryDF = null_summaryDF[null_summaryDF['Nulls'] != 0].sort_values(by=['Nulls'],ascending=False)
    return summaryDF

In [868]:
#statRow = []
cleanActionsDF = pd.DataFrame(columns = ['DF','ActionDesc','List-of-Cols','ResultShape'])

def cleanActionsReport (df,df_name,actionDesc, cols):
    # Collect Stats
    row =[]
    actionDesc = actionDesc
    listCols = cols
    resultShape = df.shape
    # Collect Results
    row = [df_name,actionDesc, listCols, resultShape]
    # append stats to Summary Stats DF
    df = cleanActionsDF
    df_length = len(df)
    df.loc[df_length] = row
    return df

In [869]:
# Dropping Missing Values over 30% -- Will print out a table that shows what variables are dropped
def dropCols_nullThresh(df,thresh,df_name,actionDesc):
    thresh = 1-thresh
    column_names_before = df.columns.to_list()
    before = df.shape[1]
    df = df.dropna(thresh=df.shape[0]*thresh,how='all',axis=1)
    after = df.shape[1]
    column_names_after = df.columns.to_list()
    columns_dropped = []
    for i in column_names_before:
        if i not in column_names_after:
            columns_dropped.append(i)
    total_dropped = before - after
    df = df
    print(f"dropped: {total_dropped}")
    print(f"columns that where dropped where: {columns_dropped}")
    # record actions in summary report
    cleanActionsReport(df,df_name,actionDesc,columns_dropped)
    return df

In [870]:
# Dropping Missing Values over 30% -- Will print out a table that shows what variables are dropped
def dropCols_uniqueThresh(df,thresh,df_name,actionDesc):
    thresh = thresh
    column_names_before = df.columns.to_list()
    before = df.shape[1]
    df = df.drop(columns=df.columns[df.nunique()==1])
    after = df.shape[1]
    column_names_after = df.columns.to_list()
    columns_dropped = []
    for i in column_names_before:
        if i not in column_names_after:
            columns_dropped.append(i)
    total_dropped = before - after
    df = df
    print(f"dropped: {total_dropped}")
    print(f"columns that where dropped where: {columns_dropped}")
    # record actions in summary report
    cleanActionsReport(df,df_name,actionDesc,columns_dropped)
    return df

In [871]:
# Takes a DF, list of variable,and an order and returns a DF w/ ordinal encoding
def OrdinalEncoderFunc (df,listOfVars,order):
    enc = OrdinalEncoder(categories=[order])
    for col in listOfVars:
        col=col
        df[col] = enc.fit_transform(df[[col]])
    return df

# Source Data

## Prep -- Trim Large CSV
(Note: Creates trimmed version of CSV's for import.
Only need to run this once, and then comment out to save time)



In [872]:
# Trim Fundamentals_Full
largeCSVPath = 'C:/Users/TheCu/OneDrive/Documents/Grad-School-Docs/CapstoneProject/SourceData_Orig/'
CSVName = 'Fundamentals_Full.csv'
trimQuery = "gsector == 30"

print("Starting...")
#fundamentals = pd.read_csv(largeCSVPath+CSVName)
#fundamentals['gsector'] = fundamentals['gsector'].astype(str)
#fundamentals = fundamentals[fundamentals['gsector'] == '30']
#fundamentals.to_csv(trimmedCSVPath+'Fundamentals_30.csv') # Export Trimmed Data

#View Trimmed Dataframe
print("Fundamentals_30:")
#fundamentals.shape

Starting...
Fundamentals_30:


In [873]:
# Trim Securities_Full
largeCSVPath = 'C:/Users/TheCu/OneDrive/Documents/Grad-School-Docs/CapstoneProject/SourceData_Orig/'
CSVName = 'Securities_Full.csv'
trimQuery = "gsector == 30"

print("Starting...")
#securities_30 = pd.read_csv(largeCSVPath+CSVName)
#securities_30['gsector'] = securities_30['gsector'].astype(str)
#securities_30 = securities_30[securities_30['gsector'] == '30']
#securities_30.to_csv(trimmedCSVPath+'Securities_30.csv') # Export Trimmed Data

#View Trimmed Dataframe
print("Securities_30:")
#securities_30.shape

Starting...
Securities_30:


In [874]:
# Trim Stocks_DS
largeCSVPath = 'C:/Users/TheCu/OneDrive/Documents/Grad-School-Docs/CapstoneProject/SourceData_Orig/'
CSVName = 'Stocks_DS.csv'
trimQuery = "gsector == 30"

print("Starting...")
#stocks_30 = pd.read_csv(largeCSVPath+CSVName)
#stocks_30 = stocks_30[stocks_30['gsector'] == 30]
#stocks_30.to_csv(trimmedCSVPath+'Stocks_30.csv') # Export Trimmed Data

#View Trimmed Dataframe
print("Stocks_30:")
#stocks_30.sample(3)

Starting...
Stocks_30:


## Get Source Data
(Note: Creates trimmed version of CSV's for import.
Only need to run this once, and then comment out to save time)

In [875]:
# Get Fundamentals
file = 'Fundamentals_30.csv'
fundamentals = pd.read_csv(sourceDataPath+file,skipinitialspace = True)
fundamentals.head()


,Unnamed: 0,gvkey,datadate,fyear,indfmt,consol,popsrc,datafmt,tic,conm,...,priusa,sic,spcindcd,spcseccd,spcsrc,state,stko,weburl,dldte,ipodate
0,218,1239,9/30/2010,2010,INDL,C,D,SUMM_STD,ACV,ALBERTO-CULVER CO,...,1,2844,215.0,978.0,A-,IL,0,www.alberto.com,5/11/2011,NaN
1,219,1239,9/30/2010,2010,INDL,C,D,STD,ACV,ALBERTO-CULVER CO,...,1,2844,215.0,978.0,A-,IL,0,www.alberto.com,5/11/2011,NaN
2,242,1266,9/30/2010,2010,INDL,C,D,SUMM_STD,ALCO,ALICO INC,...,1,100,112.0,970.0,B-,FL,0,www.alicoinc.com,NaN,NaN
3,243,1266,9/30/2010,2010,INDL,C,D,STD,ALCO,ALICO INC,...,1,100,112.0,970.0,B-,FL,0,www.alicoinc.com,NaN,NaN
4,244,1266,9/30/2011,2011,INDL,C,D,SUMM_STD,ALCO,ALICO INC,...,1,100,112.0,970.0,B-,FL,0,www.alicoinc.com,NaN,NaN


In [876]:
# Get Fundmentals Data Dictionary
file = 'DataDictionary.xlsx'

fundamentals_DD = pd.read_excel(sourceDataPath+file,sheet_name='Fundamentals')
fundamentals_DD.head()

,Mnemonic,Definition
0,gvkey,Global Company Key
1,datadate,Data Date
2,fyear,Data Year - Fiscal
3,indfmt,Industry Format
4,consol,Level of Consolidation - Company Annual Descri...


In [877]:
# Get Ratings

file = 'Ratings_DS.csv'
ratings = pd.read_csv(sourceDataPath+file,skipinitialspace = True)
ratings.head()

,gvkey,splticrm,spsdrm,spsticrm,datadate,city,conml,ggroup,gind,gsector,...,idbflag,loc,naics,sic,spcindcd,spcseccd,spcsrc,state,conm,tic
0,1003,NaN,NaN,NaN,1/31/2010,St. Louis,A.A. Importing Co Inc,2550,255040,25,...,D,USA,442110,5712,449.0,976.0,NaN,MO,A.A. IMPORTING CO INC,ANTQ
1,1003,NaN,NaN,NaN,2/28/2010,St. Louis,A.A. Importing Co Inc,2550,255040,25,...,D,USA,442110,5712,449.0,976.0,NaN,MO,A.A. IMPORTING CO INC,ANTQ
2,1003,NaN,NaN,NaN,3/31/2010,St. Louis,A.A. Importing Co Inc,2550,255040,25,...,D,USA,442110,5712,449.0,976.0,NaN,MO,A.A. IMPORTING CO INC,ANTQ
3,1003,NaN,NaN,NaN,4/30/2010,St. Louis,A.A. Importing Co Inc,2550,255040,25,...,D,USA,442110,5712,449.0,976.0,NaN,MO,A.A. IMPORTING CO INC,ANTQ
4,1003,NaN,NaN,NaN,5/31/2010,St. Louis,A.A. Importing Co Inc,2550,255040,25,...,D,USA,442110,5712,449.0,976.0,NaN,MO,A.A. IMPORTING CO INC,ANTQ


In [878]:
# Get Securities

file = 'Securities_30.csv'
securities = pd.read_csv(sourceDataPath+file,skipinitialspace = True)
securities.head()

,Unnamed: 0,gvkey,iid,datadate,tic,conm,ajexm,ajpm,isalrt,primiss,...,conml,costat,ggroup,gind,gsector,gsubind,loc,naics,sic,state
0,1984,1239,01,1/31/2010,ACV,ALBERTO-CULVER CO,1.0,1.0,NaN,P,...,Alberto-Culver Co,I,3030,303020,30,30302010,USA,325620,2844,IL
1,1985,1239,01,2/28/2010,ACV,ALBERTO-CULVER CO,1.0,1.0,NaN,P,...,Alberto-Culver Co,I,3030,303020,30,30302010,USA,325620,2844,IL
2,1986,1239,01,3/31/2010,ACV,ALBERTO-CULVER CO,1.0,1.0,NaN,P,...,Alberto-Culver Co,I,3030,303020,30,30302010,USA,325620,2844,IL
3,1987,1239,01,4/30/2010,ACV,ALBERTO-CULVER CO,1.0,1.0,NaN,P,...,Alberto-Culver Co,I,3030,303020,30,30302010,USA,325620,2844,IL
4,1988,1239,01,5/31/2010,ACV,ALBERTO-CULVER CO,1.0,1.0,NaN,P,...,Alberto-Culver Co,I,3030,303020,30,30302010,USA,325620,2844,IL


In [879]:
# Get Securities Data Dictionary
file = 'DataDictionary.xlsx'

securities_DD = pd.read_excel(sourceDataPath+file,sheet_name='Securities')
securities_DD.head()

,Mnemonic,Definition
0,gvkey,Global Company Key
1,iid,Issue ID - Security Monthly Descriptor
2,datadate,Data Date - Security Monthly
3,tic,Ticker Symbol
4,conm,Company Name


In [880]:
# Get Stocks
file = 'Stocks_30.csv'
stocks = pd.read_csv(sourceDataPath+file,skipinitialspace = True)
stocks.head()

,Unnamed: 0,gvkey,iid,datadate,tic,cusip,conm,curcddv,capgn,cheqv,...,priusa,sic,spcindcd,spcseccd,spcsrc,state,stko,weburl,dldte,ipodate
0,9739,1239,1,1/4/2010,ACV,013078100,ALBERTO-CULVER CO,NaN,NaN,NaN,...,1.0,2844,215.0,978.0,A-,IL,0.0,www.alberto.com,5/11/2011,NaN
1,9740,1239,1,1/5/2010,ACV,013078100,ALBERTO-CULVER CO,NaN,NaN,NaN,...,1.0,2844,215.0,978.0,A-,IL,0.0,www.alberto.com,5/11/2011,NaN
2,9741,1239,1,1/6/2010,ACV,013078100,ALBERTO-CULVER CO,NaN,NaN,NaN,...,1.0,2844,215.0,978.0,A-,IL,0.0,www.alberto.com,5/11/2011,NaN
3,9742,1239,1,1/7/2010,ACV,013078100,ALBERTO-CULVER CO,NaN,NaN,NaN,...,1.0,2844,215.0,978.0,A-,IL,0.0,www.alberto.com,5/11/2011,NaN
4,9743,1239,1,1/8/2010,ACV,013078100,ALBERTO-CULVER CO,NaN,NaN,NaN,...,1.0,2844,215.0,978.0,A-,IL,0.0,www.alberto.com,5/11/2011,NaN


In [881]:
# Get Stocks Data Dictionary
file = 'DataDictionary.xlsx'

stocks_DD = pd.read_excel(sourceDataPath+file,sheet_name='Stocks')
stocks_DD.head()

,Mnemonic,Definition
0,gvkey,Global Company Key
1,iid,Issue ID - Dividends
2,datadate,Data Date - Dividends
3,tic,Ticker Symbol
4,cusip,CUSIP


In [882]:
# Get SCA Filings
file = 'SCA Filings and Settlements.xlsx'

sca_filings = pd.read_excel(sourceDataPath+file,sheet_name='SCA Filings and Settlements')
sca_filings.head()

,FilingName,Exchange,Ticker,Dismissed,SettlementAmount,FilingYear
0,"AAC Holdings, Inc.",New York SE,AAC,Ongoing,#NULL!,2015
1,"Airtran Holdings, Inc.",New York SE,AAI,Yes,#NULL!,2011
2,Altisource Asset Management Corporation,New York SE,AAMC,Ongoing,#NULL!,2015
3,"APPLE Computer, Inc.",NASDAQ,AAPL,No,16500000,2006
4,"Avalanche Biotechnologies, Inc.",NASDAQ,AAVL,Ongoing,#NULL!,2015


# Data Clean-Up

## Fundamentals

In [883]:
fund_nunique = fundamentals['gvkey'].nunique()
fund_nunique

348

In [884]:
# Drop Obvious Un-needed 1
cleanActionsDesc = "Dropped obvious low value columns -- 1 -- Dates/Codes/Addresses/Etc."
df_name = "Fundamentals"

colsToDrop = ['datadate','apdedate','fdate','pdate','fyr','add1','addzip','busdesc','city','conml',
              'ein','fax','fyrc','incorp','loc','phone','state','weburl','exchg','naicsh','sich','gsubind',
              'naics','sic','stko']

fundamentals = fundamentals.drop(columns=colsToDrop)
cleanActionsReport(fundamentals,df_name,cleanActionsDesc,colsToDrop)

,DF,ActionDesc,List-of-Cols,ResultShape
0,Fundamentals,Dropped obvious low value columns -- 1 -- Date...,"[datadate, apdedate, fdate, pdate, fyr, add1, ...","(2323, 1744)"


In [885]:
# Initial Column Reduce -- Drop Obvious -- Nulls
# Run Initial -- Null Report
fundamentals_nullReport = nullAnalysis(fundamentals)
fundamentals_nullReport

Count of Variables: 1744
Count of Variables without nulls: 20
Count of Variables with null : 1724


,Variable,Observations,Nulls,Null_Per,Num_Unique,Type
875,xstf,2323,2323,100.000,0,float64
1385,isfxc_dc,2323,2323,100.000,0,float64
703,tdc,2323,2323,100.000,0,float64
704,tdscd,2323,2323,100.000,0,float64
705,tdsce,2323,2323,100.000,0,float64
...,...,...,...,...,...,...
346,ib,2323,67,2.884,1336,float64
713,teq,2323,60,2.583,1316,float64
88,at,2323,58,2.497,1310,float64
669,sale,2323,58,2.497,1328,float64


In [886]:
# Initial Column Reduce -- Drop Obvious -- Nulls
# Initial Null Drop -- More than 75% -- Will Re-Run w/ lower threshold after rows combined
cleanActionsDesc = "Initial Null Drop -- columns w/ 63% Null or more."
df_name = "Fundamentals"

fundamentals = dropCols_nullThresh(fundamentals,0.63,df_name,cleanActionsDesc)
fundamentals

dropped: 1418
columns that where dropped where: ['acctchg', 'acqmeth', 'adrr', 'bspr', 'compst', 'curuscn', 'ltcm', 'ogm', 'stalt', 'udpl', 'acco', 'acoxar', 'acqao', 'acqcshi', 'acqgdwl', 'acqic', 'acqintan', 'acqinvt', 'acqlntal', 'acqniintc', 'acqppe', 'acqsc', 'adpac', 'aedi', 'afudcc', 'afudci', 'amc', 'amdc', 'amgw', 'ano', 'aol2', 'apalch', 'apb', 'apc', 'apofs', 'aqa', 'aqd', 'aqeps', 'aqi', 'aqp', 'aqpl1', 'aqs', 'arb', 'arc', 'arce', 'arced', 'arceeps', 'artfs', 'aul3', 'autxr', 'balr', 'banlr', 'bast', 'bastr', 'batr', 'bcef', 'bclr', 'bcltbl', 'bcnlr', 'bcrbl', 'bct', 'bctbl', 'bctr', 'bltbl', 'ca', 'capr1', 'capr2', 'capr3', 'cb', 'cbi', 'cdpac', 'cdvc', 'cfbd', 'cfere', 'cfo', 'cfpdo', 'cga', 'cgri', 'cgti', 'cgui', 'chs', 'clfc', 'clfx', 'clg', 'clis', 'cll', 'cllc', 'clo', 'clrll', 'clt', 'cmp', 'cnltbl', 'cpcbl', 'cpdoi', 'cpnli', 'cppbl', 'cprei', 'crv', 'crvnli', 'cshrc', 'cshrp', 'cshrso', 'cshrt', 'cshrw', 'dbi', 'dcs', 'depc', 'dfpac', 'dfs', 'dfxa', 'dlcch', 'dlt

,Unnamed: 0,gvkey,fyear,indfmt,consol,popsrc,datafmt,tic,conm,acctstd,...,rdipeps_fn,stkco_fn,ggroup,gind,gsector,idbflag,priusa,spcindcd,spcseccd,spcsrc
0,218,1239,2010,INDL,C,D,SUMM_STD,ACV,ALBERTO-CULVER CO,NaN,...,NaN,NaN,3030,303020,30,D,1,215.0,978.0,A-
1,219,1239,2010,INDL,C,D,STD,ACV,ALBERTO-CULVER CO,DS,...,NR,OE,3030,303020,30,D,1,215.0,978.0,A-
2,242,1266,2010,INDL,C,D,SUMM_STD,ALCO,ALICO INC,NaN,...,NaN,NaN,3020,302020,30,D,1,112.0,970.0,B-
3,243,1266,2010,INDL,C,D,STD,ALCO,ALICO INC,DS,...,NR,OE,3020,302020,30,D,1,112.0,970.0,B-
4,244,1266,2011,INDL,C,D,SUMM_STD,ALCO,ALICO INC,NaN,...,NaN,NaN,3020,302020,30,D,1,112.0,970.0,B-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2318,60832,264393,2013,INDL,C,D,SUMM_STD,IVFH,INNOVATIVE FOOD HOLDINGS,NaN,...,NaN,NaN,3010,301010,30,D,1,NaN,NaN,NaN
2319,60833,264393,2013,INDL,C,D,STD,IVFH,INNOVATIVE FOOD HOLDINGS,DS,...,NR,OE,3010,301010,30,D,1,NaN,NaN,NaN
2320,61135,277487,2011,INDL,C,D,STD,LND,BRASILAGRO CIA BRAS DE PROP,DI,...,NR,OE,3020,302020,30,B,90,NaN,NaN,NaN
2321,61136,277487,2012,INDL,C,D,STD,LND,BRASILAGRO CIA BRAS DE PROP,DI,...,NR,OE,3020,302020,30,B,90,NaN,NaN,NaN


In [887]:
# Initial Column Reduce -- Drop Obvious -- Nulls
# Re-Run Null Report
fundamentals_nullReport = nullAnalysis(fundamentals)
fundamentals_nullReport

Count of Variables: 326
Count of Variables without nulls: 20
Count of Variables with null : 306


,Variable,Observations,Nulls,Null_Per,Num_Unique,Type
264,txds,2323,1462,62.936,443,float64
114,dxd5,2323,1459,62.807,318,float64
260,txdfed,2323,1458,62.764,547,float64
196,optex,2323,1457,62.721,644,float64
152,invfg,2323,1454,62.591,769,float64
...,...,...,...,...,...,...
141,ib,2323,67,2.884,1336,float64
247,teq,2323,60,2.583,1316,float64
40,at,2323,58,2.497,1310,float64
235,sale,2323,58,2.497,1328,float64


In [888]:
# Initial Column Reduce -- Drop Obvious -- Cols w/ 1 Value
cleanActionsDesc = "Dropped columns with only 1 unique value "
df_name = "Fundamentals"

fundamentals = dropCols_uniqueThresh(fundamentals,1,df_name,cleanActionsDesc)
fundamentals


dropped: 16
columns that where dropped where: ['indfmt', 'consol', 'popsrc', 'curcd', 'final', 'scf', 'acchg', 'aocisecgl', 'esopr', 'itcb', 'xoptd', 'xopteps', 'rank', 'rdipa_fn', 'stkco_fn', 'gsector']


,Unnamed: 0,gvkey,fyear,datafmt,tic,conm,acctstd,ajex,ajp,curncd,...,dpact_fn,rdipd_fn,rdipeps_fn,ggroup,gind,idbflag,priusa,spcindcd,spcseccd,spcsrc
0,218,1239,2010,SUMM_STD,ACV,ALBERTO-CULVER CO,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,3030,303020,D,1,215.0,978.0,A-
1,219,1239,2010,STD,ACV,ALBERTO-CULVER CO,DS,1.0,1.0,USD,...,TS,NR,NR,3030,303020,D,1,215.0,978.0,A-
2,242,1266,2010,SUMM_STD,ALCO,ALICO INC,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,3020,302020,D,1,112.0,970.0,B-
3,243,1266,2010,STD,ALCO,ALICO INC,DS,1.0,1.0,USD,...,TB,NR,NR,3020,302020,D,1,112.0,970.0,B-
4,244,1266,2011,SUMM_STD,ALCO,ALICO INC,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,3020,302020,D,1,112.0,970.0,B-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2318,60832,264393,2013,SUMM_STD,IVFH,INNOVATIVE FOOD HOLDINGS,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,3010,301010,D,1,NaN,NaN,NaN
2319,60833,264393,2013,STD,IVFH,INNOVATIVE FOOD HOLDINGS,DS,1.0,1.0,USD,...,TS,NR,NR,3010,301010,D,1,NaN,NaN,NaN
2320,61135,277487,2011,STD,LND,BRASILAGRO CIA BRAS DE PROP,DI,1.0,1.0,BRR,...,TS,NR,NR,3020,302020,B,90,NaN,NaN,NaN
2321,61136,277487,2012,STD,LND,BRASILAGRO CIA BRAS DE PROP,DI,1.0,1.0,BRR,...,TS,NR,NR,3020,302020,B,90,NaN,NaN,NaN


In [889]:
cleanActionsDF

,DF,ActionDesc,List-of-Cols,ResultShape
0,Fundamentals,Dropped obvious low value columns -- 1 -- Date...,"[datadate, apdedate, fdate, pdate, fyr, add1, ...","(2323, 1744)"
1,Fundamentals,Initial Null Drop -- columns w/ 63% Null or more.,"[acctchg, acqmeth, adrr, bspr, compst, curuscn...","(2323, 326)"
2,Fundamentals,Dropped columns with only 1 unique value,"[indfmt, consol, popsrc, curcd, final, scf, ac...","(2323, 310)"


### Fundamentals -- Restatements

In [890]:
# make table w/ just restatement rows to get meaningful cols for a diff
## note came from a lengthy analysis -- analysis docs stored in repo
fund_restatement = fundamentals[['gvkey','fyear','datafmt','tic',
                                 'ni','revt','epsfi','epspi','cogs',
                                 'spce','xopr','gdwl','teq', 'mii',
                                 'nopi','pi','txt','wcap','xsga']]

# Make copy w/ cols to rejoin after diff
fund_restatement_rejoin = fund_restatement[['gvkey','fyear','datafmt','tic']]
# Drop string columns that can't be used for diff
fund_restatement = fund_restatement.drop(columns =['datafmt','tic'])
fund_restatement

,gvkey,fyear,ni,revt,epsfi,epspi,cogs,spce,xopr,gdwl,teq,mii,nopi,pi,txt,wcap,xsga
0,1239,2010,155.310,NaN,1.56,1.59,734.501,NaN,NaN,NaN,1324.612,0.000,-13.684,217.844,62.708,520.680,601.963
1,1239,2010,155.310,1598.464,1.56,1.59,734.501,155.053,1336.464,521.204,1327.240,0.000,1.490,217.844,62.708,520.680,601.963
2,1266,2010,-0.623,NaN,-0.08,-0.08,62.648,NaN,NaN,NaN,105.237,0.000,-1.412,-1.824,-1.201,29.529,6.458
3,1266,2010,-0.623,79.792,-0.08,-0.08,62.648,-0.936,69.106,0.000,105.237,0.000,1.688,-1.824,-1.201,29.529,6.458
4,1266,2011,7.097,NaN,0.96,0.96,67.832,NaN,NaN,NaN,110.662,0.000,-0.563,12.527,5.430,17.354,8.196
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2318,264393,2013,-1.486,NaN,-0.23,-0.23,16.854,NaN,NaN,NaN,0.940,0.000,0.000,-1.486,0.000,-0.174,5.420
2319,264393,2013,-1.486,23.503,-0.23,-0.23,16.854,-1.486,22.274,0.151,0.940,0.000,0.000,-1.486,0.000,-0.174,5.420
2320,277487,2011,9.454,51.006,0.16,0.16,30.511,9.425,48.677,0.000,369.941,-0.051,30.513,12.728,3.325,88.979,18.166
2321,277487,2012,-2.774,79.246,-0.05,-0.05,54.097,-2.777,69.916,0.000,278.232,-0.511,17.423,-9.679,-6.394,53.942,15.819


In [891]:
# Run Diff
fund_restatement_diff = fund_restatement.diff()
fund_restatement_diff = fund_restatement_diff.rename(columns={'gvkey':'gvkey_diff','fyear':'fyear_diff'})
fund_restatement_diff = fund_restatement_rejoin.join(fund_restatement_diff)
fund_restatement_diff


,gvkey,fyear,datafmt,tic,gvkey_diff,fyear_diff,ni,revt,epsfi,epspi,...,spce,xopr,gdwl,teq,mii,nopi,pi,txt,wcap,xsga
0,1239,2010,SUMM_STD,ACV,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1239,2010,STD,ACV,0.0,0.0,0.000,NaN,0.00,0.00,...,NaN,NaN,NaN,2.628,0.000,15.174,0.000,0.000,0.000,0.000
2,1266,2010,SUMM_STD,ALCO,27.0,0.0,-155.933,NaN,-1.64,-1.67,...,NaN,NaN,NaN,-1222.003,0.000,-2.902,-219.668,-63.909,-491.151,-595.505
3,1266,2010,STD,ALCO,0.0,0.0,0.000,NaN,0.00,0.00,...,NaN,NaN,NaN,0.000,0.000,3.100,0.000,0.000,0.000,0.000
4,1266,2011,SUMM_STD,ALCO,0.0,1.0,7.720,NaN,1.04,1.04,...,NaN,NaN,NaN,5.425,0.000,-2.251,14.351,6.631,-12.175,1.738
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2318,264393,2013,SUMM_STD,IVFH,0.0,1.0,-3.516,NaN,-0.48,-0.59,...,NaN,NaN,NaN,-0.281,0.000,1.282,-3.516,0.000,-1.082,1.315
2319,264393,2013,STD,IVFH,0.0,0.0,0.000,NaN,0.00,0.00,...,NaN,NaN,NaN,0.000,0.000,0.000,0.000,0.000,0.000,0.000
2320,277487,2011,STD,LND,13094.0,-2.0,10.940,27.503,0.39,0.39,...,10.911,26.403,-0.151,369.001,-0.051,30.513,14.214,3.325,89.153,12.746
2321,277487,2012,STD,LND,0.0,1.0,-12.228,28.240,-0.21,-0.21,...,-12.202,21.239,0.000,-91.709,-0.460,-13.090,-22.407,-9.719,-35.037,-2.347


In [892]:
# Trim Diff -- only rows that compare same gvkey and same year
fund_restatement_diff = fund_restatement_diff[fund_restatement_diff['gvkey_diff'] == 0]
fund_restatement_diff = fund_restatement_diff[fund_restatement_diff['fyear_diff'] == 0]

# Add Sum Column
#TODO: Make Diffs Absolute so positive/negative don't cancel each other
colsToSum = ['ni','revt','epsfi','epspi','cogs', # "A-LIST Identifiers
             'spce','xopr','gdwl','teq', 'mii','nopi','pi','txt','wcap','xsga']

fund_restatement_diff['sum_diff'] = fund_restatement_diff[colsToSum].sum(axis=1)

#Drop Columns Where Sum of Diffs = 0
fund_restatement_diff = fund_restatement_diff[fund_restatement_diff['sum_diff'] > 0]
fund_restatement_diff

,gvkey,fyear,datafmt,tic,gvkey_diff,fyear_diff,ni,revt,epsfi,epspi,...,xopr,gdwl,teq,mii,nopi,pi,txt,wcap,xsga,sum_diff
1,1239,2010,STD,ACV,0.0,0.0,0.0,NaN,0.00,0.00,...,NaN,NaN,2.628,0.0,15.174,0.0,0.0,0.0,0.0,17.802
3,1266,2010,STD,ALCO,0.0,0.0,0.0,NaN,0.00,0.00,...,NaN,NaN,0.000,0.0,3.100,0.0,0.0,0.0,0.0,3.100
5,1266,2011,STD,ALCO,0.0,0.0,0.0,NaN,0.00,0.00,...,NaN,NaN,0.000,0.0,1.685,0.0,0.0,0.0,0.0,1.685
7,1266,2012,STD,ALCO,0.0,0.0,0.0,NaN,0.00,0.00,...,NaN,NaN,0.000,0.0,1.918,0.0,0.0,0.0,0.0,1.918
11,1408,2010,STD,BEAM,0.0,0.0,0.0,NaN,0.00,0.00,...,NaN,NaN,-3.000,8.4,15.800,241.9,55.4,NaN,1325.1,4341.100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2272,199456,2013,STD,RNDY,0.0,0.0,0.0,NaN,0.00,0.00,...,NaN,NaN,0.000,0.0,2.494,0.0,0.0,0.0,0.0,2.494
2305,241637,2010,STD,BUD,0.0,0.0,0.0,NaN,0.00,0.00,...,NaN,NaN,0.000,0.0,1193.000,0.0,0.0,0.0,0.0,1193.000
2307,241637,2011,STD,BUD,0.0,0.0,0.0,NaN,0.00,0.00,...,NaN,NaN,-12.000,0.0,818.000,0.0,0.0,0.0,0.0,806.000
2309,241637,2012,STD,BUD,0.0,0.0,83.0,NaN,0.05,0.05,...,NaN,NaN,-12.000,26.0,50.000,146.0,37.0,0.0,-171.0,184.100


In [893]:
a_colsToSum = ['ni','revt','epsfi','epspi','cogs'] # "A-LIST Identifiers
fund_restatement_diff['a_sum_diff'] = fund_restatement_diff[a_colsToSum].sum(axis=1)
fund_restatement_diff

,gvkey,fyear,datafmt,tic,gvkey_diff,fyear_diff,ni,revt,epsfi,epspi,...,gdwl,teq,mii,nopi,pi,txt,wcap,xsga,sum_diff,a_sum_diff
1,1239,2010,STD,ACV,0.0,0.0,0.0,NaN,0.00,0.00,...,NaN,2.628,0.0,15.174,0.0,0.0,0.0,0.0,17.802,0.00
3,1266,2010,STD,ALCO,0.0,0.0,0.0,NaN,0.00,0.00,...,NaN,0.000,0.0,3.100,0.0,0.0,0.0,0.0,3.100,0.00
5,1266,2011,STD,ALCO,0.0,0.0,0.0,NaN,0.00,0.00,...,NaN,0.000,0.0,1.685,0.0,0.0,0.0,0.0,1.685,0.00
7,1266,2012,STD,ALCO,0.0,0.0,0.0,NaN,0.00,0.00,...,NaN,0.000,0.0,1.918,0.0,0.0,0.0,0.0,1.918,0.00
11,1408,2010,STD,BEAM,0.0,0.0,0.0,NaN,0.00,0.00,...,NaN,-3.000,8.4,15.800,241.9,55.4,NaN,1325.1,4341.100,2697.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2272,199456,2013,STD,RNDY,0.0,0.0,0.0,NaN,0.00,0.00,...,NaN,0.000,0.0,2.494,0.0,0.0,0.0,0.0,2.494,0.00
2305,241637,2010,STD,BUD,0.0,0.0,0.0,NaN,0.00,0.00,...,NaN,0.000,0.0,1193.000,0.0,0.0,0.0,0.0,1193.000,0.00
2307,241637,2011,STD,BUD,0.0,0.0,0.0,NaN,0.00,0.00,...,NaN,-12.000,0.0,818.000,0.0,0.0,0.0,0.0,806.000,0.00
2309,241637,2012,STD,BUD,0.0,0.0,83.0,NaN,0.05,0.05,...,NaN,-12.000,26.0,50.000,146.0,37.0,0.0,-171.0,184.100,108.10


In [894]:
# Create Diff Summary
fund_restatement_summary = fund_restatement_diff

## Add Column that counts non-zero's per row
fund_restatement_summary['count_of_diffs'] = fund_restatement_summary[colsToSum].gt(0).sum(axis=1)
fund_restatement_summary['a_count_of_diffs'] = fund_restatement_summary[a_colsToSum].gt(0).sum(axis=1)

# Drop Individual Counts, _diff columns, tic,
fund_restatement_summary = fund_restatement_summary.drop(columns=['datafmt','tic','gvkey_diff','fyear_diff',
                                                                  'cogs','mii','nopi','pi','txt','wcap','xsga'])

# Group By -- Turn Year into count of years
fund_restatement_summary = fund_restatement_summary.groupby('gvkey')\
    .agg({'fyear':'count','sum_diff':'sum', 'count_of_diffs': 'sum',
          'a_sum_diff':'sum','a_count_of_diffs': 'sum'}).reset_index()
fund_restatement_summary = fund_restatement_summary.rename(columns={'fyear':'count'})
fund_restatement_summary

,gvkey,count,sum_diff,count_of_diffs,a_sum_diff,a_count_of_diffs
0,1239,1,17.802,2,0.000,0
1,1266,3,6.703,3,0.000,0
2,1408,3,4799.820,14,2708.520,5
3,1429,4,4.960,5,4.897,4
4,1659,2,7.844,3,1.644,1
...,...,...,...,...,...,...
236,196258,1,0.022,3,0.017,1
237,197956,4,254.256,26,1.588,9
238,199456,3,139.671,6,0.160,4
239,241637,3,2183.100,10,108.100,4


In [895]:
# rename for clarity

fund_restatement_summary = fund_restatement_summary.add_prefix(prefix="rest_")
fund_restatement_summary = fund_restatement_summary.rename(columns={'rest_gvkey':'gvkey'})
fund_restatement_summary

,gvkey,rest_count,rest_sum_diff,rest_count_of_diffs,rest_a_sum_diff,rest_a_count_of_diffs
0,1239,1,17.802,2,0.000,0
1,1266,3,6.703,3,0.000,0
2,1408,3,4799.820,14,2708.520,5
3,1429,4,4.960,5,4.897,4
4,1659,2,7.844,3,1.644,1
...,...,...,...,...,...,...
236,196258,1,0.022,3,0.017,1
237,197956,4,254.256,26,1.588,9
238,199456,3,139.671,6,0.160,4
239,241637,3,2183.100,10,108.100,4


In [896]:
# Export Diff Reports to Excel

with pd.ExcelWriter(exportPath + 'Fundamentals_Restatement_Diff_Reports.xlsx') as writer:
    fund_restatement_diff.to_excel(writer, sheet_name='All_Diffs')
    fund_restatement_summary.to_excel(writer, sheet_name='Summary')


### Fundamentals -- Resume Data Clean-up
Many columns could not be dropped until restatement was calculated

In [897]:
# drop restatement rows
## no longer useful since restatement data has been summarized
fundamentals = fundamentals[fundamentals['datafmt'] == 'STD']

cleanActionsDesc = 'Dropping all that are not standard statements -- value of others captured in restatement summary'
df_name = 'Fundamentals'
cols = 'applies to all cols, roughly 50% of rows dropped'

cleanActionsReport(fundamentals,df_name,cleanActionsDesc, cols)

,DF,ActionDesc,List-of-Cols,ResultShape
0,Fundamentals,Dropped obvious low value columns -- 1 -- Date...,"[datadate, apdedate, fdate, pdate, fyr, add1, ...","(2323, 1744)"
1,Fundamentals,Initial Null Drop -- columns w/ 63% Null or more.,"[acctchg, acqmeth, adrr, bspr, compst, curuscn...","(2323, 326)"
2,Fundamentals,Dropped columns with only 1 unique value,"[indfmt, consol, popsrc, curcd, final, scf, ac...","(2323, 310)"
3,Fundamentals,Dropping all that are not standard statements ...,"applies to all cols, roughly 50% of rows dropped","(1243, 310)"


In [898]:
# Second Column Reduce -- Nulls
# Second Null Drop -- More than 20%
cleanActionsDesc = "Second Null Drop -- columns w/ 20% Null or more."
df_name = "Fundamentals"

fundamentals = dropCols_nullThresh(fundamentals,0.20,df_name,cleanActionsDesc)
fundamentals


dropped: 45
columns that where dropped where: ['cld2', 'cld3', 'cld4', 'cld5', 'cshr', 'dd3', 'dd4', 'dd5', 'dltp', 'dxd2', 'dxd3', 'dxd4', 'dxd5', 'fatb', 'fate', 'fato', 'fatp', 'invfg', 'invo', 'itci', 'mrc2', 'mrc3', 'mrc4', 'mrc5', 'mrct', 'optca', 'optex', 'optexd', 'optgr', 'optosby', 'optosey', 'recd', 'sppe', 'stkco', 'txc', 'txdfed', 'txdfo', 'txds', 'txfed', 'txs', 'xrent', 'mkvalt', 'spcindcd', 'spcseccd', 'spcsrc']


,Unnamed: 0,gvkey,fyear,datafmt,tic,conm,acctstd,ajex,ajp,curncd,...,auopic,ceoso,cfoso,dpact_fn,rdipd_fn,rdipeps_fn,ggroup,gind,idbflag,priusa
1,219,1239,2010,STD,ACV,ALBERTO-CULVER CO,DS,1.0,1.0,USD,...,1.0,Y,Y,TS,NR,NR,3030,303020,D,1
3,243,1266,2010,STD,ALCO,ALICO INC,DS,1.0,1.0,USD,...,1.0,Y,Y,TB,NR,NR,3020,302020,D,1
5,245,1266,2011,STD,ALCO,ALICO INC,DS,1.0,1.0,USD,...,1.0,Y,Y,TS,NR,NR,3020,302020,D,1
7,247,1266,2012,STD,ALCO,ALICO INC,DS,1.0,1.0,USD,...,1.0,Y,Y,TS,NR,NR,3020,302020,D,1
8,248,1266,2013,STD,ALCO,ALICO INC,DS,1.0,1.0,USD,...,1.0,Y,Y,TS,NR,NR,3020,302020,D,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2317,60831,264393,2012,STD,IVFH,INNOVATIVE FOOD HOLDINGS,DS,1.0,1.0,USD,...,0.0,Y,Y,TS,NR,NR,3010,301010,D,1
2319,60833,264393,2013,STD,IVFH,INNOVATIVE FOOD HOLDINGS,DS,1.0,1.0,USD,...,0.0,Y,Y,TS,NR,NR,3010,301010,D,1
2320,61135,277487,2011,STD,LND,BRASILAGRO CIA BRAS DE PROP,DI,1.0,1.0,BRR,...,0.0,E,E,TS,NR,NR,3020,302020,B,90
2321,61136,277487,2012,STD,LND,BRASILAGRO CIA BRAS DE PROP,DI,1.0,1.0,BRR,...,0.0,E,E,TS,NR,NR,3020,302020,B,90


In [899]:
# Second Column Reduce -- Only 2 unique Values

lowValCountDF = pd.DataFrame(columns = ['Column','Count','ValueCounts'])
cols = fundamentals.columns.values.tolist()

for col in cols:
    count = fundamentals[col].nunique()
    column = col
    tempVal = fundamentals[col].value_counts(dropna=False)
    tempVal_dict = tempVal.to_dict()
    row = [column,count, tempVal_dict]
    # append stats to Summary Stats DF
    df_length = len(lowValCountDF)
    lowValCountDF.loc[df_length] = row

lowValCountDF = lowValCountDF[lowValCountDF['Count'] <= 2]
lowValCountDF = lowValCountDF.sort_values(by='Count', ascending=True)
lowValCountDF

,Column,Count,ValueCounts
3,datafmt,1,{'STD': 1243}
11,ismod,2,"{1.0: 1137, nan: 81, 3.0: 25}"
14,upd,2,"{3: 1230, 2: 13}"
179,rdip,2,"{0.0: 1202, nan: 40, -18.68: 1}"
180,rdipa,2,"{0.0: 1194, nan: 48, -12.142000000000001: 1}"
181,rdipd,2,"{0.0: 1157, nan: 85, -0.69: 1}"
182,rdipeps,2,"{0.0: 1157, nan: 85, -0.69: 1}"
207,tstkp,2,"{0.0: 1188, nan: 51, 5.1: 4}"
221,txndbr,2,"{0.0: 1120, nan: 122, 42.18600000000001: 1}"
230,xi,2,"{0.0: 1202, nan: 40, 0.16399999999999998: 1}"


In [900]:
# Second Column Reduce -- Only 2 unique Values
colsToDrop = lowValCountDF['Column'].values.tolist()
fundamentals = fundamentals.drop(columns=colsToDrop)

cleanActionsDesc = 'Dropping Columns w/ Only 2 values'
df_name = 'Fundamentals'
cols = colsToDrop

cleanActionsReport(fundamentals,df_name,cleanActionsDesc, cols)
cleanActionsDF

,DF,ActionDesc,List-of-Cols,ResultShape
0,Fundamentals,Dropped obvious low value columns -- 1 -- Date...,"[datadate, apdedate, fdate, pdate, fyr, add1, ...","(2323, 1744)"
1,Fundamentals,Initial Null Drop -- columns w/ 63% Null or more.,"[acctchg, acqmeth, adrr, bspr, compst, curuscn...","(2323, 326)"
2,Fundamentals,Dropped columns with only 1 unique value,"[indfmt, consol, popsrc, curcd, final, scf, ac...","(2323, 310)"
3,Fundamentals,Dropping all that are not standard statements ...,"applies to all cols, roughly 50% of rows dropped","(1243, 310)"
4,Fundamentals,Second Null Drop -- columns w/ 20% Null or more.,"[cld2, cld3, cld4, cld5, cshr, dd3, dd4, dd5, ...","(1243, 265)"
5,Fundamentals,Dropping Columns w/ Only 2 values,"[datafmt, ismod, upd, rdip, rdipa, rdipd, rdip...","(1243, 250)"


In [901]:
# Second Manual Column Reduce -- Indentified Low Value
colsToDrop = ['Unnamed: 0','conm','au','auopic','pddur','dpact_fn','ggroup','gind','fic','priusa']
fundamentals = fundamentals.drop(columns=colsToDrop)

cleanActionsDesc = 'Second Manual Column Reduce - Dropping Identified Low Value Columns'
df_name = 'Fundamentals'
cols = colsToDrop

cleanActionsReport(fundamentals,df_name,cleanActionsDesc, cols)
cleanActionsDF

,DF,ActionDesc,List-of-Cols,ResultShape
0,Fundamentals,Dropped obvious low value columns -- 1 -- Date...,"[datadate, apdedate, fdate, pdate, fyr, add1, ...","(2323, 1744)"
1,Fundamentals,Initial Null Drop -- columns w/ 63% Null or more.,"[acctchg, acqmeth, adrr, bspr, compst, curuscn...","(2323, 326)"
2,Fundamentals,Dropped columns with only 1 unique value,"[indfmt, consol, popsrc, curcd, final, scf, ac...","(2323, 310)"
3,Fundamentals,Dropping all that are not standard statements ...,"applies to all cols, roughly 50% of rows dropped","(1243, 310)"
4,Fundamentals,Second Null Drop -- columns w/ 20% Null or more.,"[cld2, cld3, cld4, cld5, cshr, dd3, dd4, dd5, ...","(1243, 265)"
5,Fundamentals,Dropping Columns w/ Only 2 values,"[datafmt, ismod, upd, rdip, rdipa, rdipd, rdip...","(1243, 250)"
6,Fundamentals,Second Manual Column Reduce - Dropping Identif...,"[Unnamed: 0, conm, au, auopic, pddur, dpact_fn...","(1243, 240)"


In [902]:
# Third Manual Column Reduce -- Indentified Low Value
colsToDrop = ['lco','lcox','lcoxdr', #Current Liabilities - kept total
              'lo','loxdr', # Liabilities - kept total nad equity total
              'mib','mibn', # Noncontrolling interest - kept total
              'ppeveb', # Prop, Plant, Equip - kept total/net
              'pstkc','pstkl','pstkn','pstkr', # Preferred Stock - kept total/redemption value
              'tstkc','tstkn', #TreasuryStock - kept total
              'che', # Cash Short Term -- kept total
              'cicurr','cidergl','cimii','ciother','cipen','cisecgl','citotal', # Comp Inc -- Kept Total and Starting
              'dpc','dpvieb', # Depreciation and Amortization (Cash Flow) - duplicated by other measures (kept)
              'dv','dvp','dvpa', # Cash Dividends - duplicated by other measures (kept)
              'ib','epspx', # Earnings Per Share - granular measures redundant to others (kept)
              'esopnr','esopt', # ESOP Obligation - kept total summary
              'ibadj', 'ibc', 'ibcom', 'ibmii',# Income Before Extraordinary Items - granular redundant measures
              'recch','recco','rectr', # Receivables - granular redundant measures
              'reuna', # Retained Earnings - kept total
              'sale', # Sales/Turnover (Net) - Always the same 'revt' (kept)
              'spced', 'spceeps', # S&P Core Earnings -- Kept total
              'cshtr_c','dvpsp_c','dvpsx_c','prcc_c','prch_c','prcl_c','adjex_c',# Calendar Year Features w/ fiscal equiv
              'acdo','aco','acodo','acox','aldo','aocidergl','aociother','aocipen',
              'aodo','aox','ap',# Assets -- kept 5 from this group that measure totals, net change, and acquistions
              'ceql', # Common Equity - Liquidation Value - same values as 'ceq'
              'intc', # Interest Capitalized - kept Net from this group
              'ivaco','ivaeq','ivao','ivch','ivst','ivstch', # Investing Activities -- kept net ('ivncf')
              'pncad','pncaeps', # Core Pension Adjustment - kept overall measure
              'prcad','prcaeps', # Core Post Retirement Adjustment - kept overall measure
              'xido','xidoc', # Extraordinary Items... - kept total
              'ajex','ajp', # Adjustment Factor (Company) - to do w/ divides, not much value
              'cshfd','cshi','csho','cstk','cstkcv','cstke', # Common Shares -- kept "...Earnings Per Share - Basic"
              'dclo','dcom','dcvsr','dcvsub','dcvt','dd','dd1','dd2','dltis','dlto','dm',
              'dn','ds','dudd', # Debt measures - kept total, short term debt measure, debt conv. to stock
              'fatc','fatc','fatn', # Property, Plant, and Equipment - didn't seem to provide value
              'fiao', # Financing Activities - kept Net cash flow
              'fopox', # Funds from Operations - kept Other
              'intano', # Intangible Assets - kept Total
              'mrc1','mrcta', # Rental Commitments - didn't seem meaningful
              'niadj', # Net Income -- duplicative of ni (kept)
              'nopio', # Nonoperating Income (Expense) -- kept total
              'oiadp','oibdp','oprepsx', # Operating Activities - kept net cash flow and earnings/share
              'pnrsho','prsho', # Pfd Shares Outs - Most were 0 or null
              'ppent', # Property, Plant and Equipment - kept Total (Gross)
              'pstkrv',# Preferred Stock - (may have more to remove) - duplicative
              'txbco','txbcof','txdba','txdbca','txdbcl','txdc','txdi','txditc','txndba','txndbl','txo','txp',
              'txpd','txr', # Taxes - kept a few total measures
              'acctstd', # Accounting Standard -- didn't look to provide value
              'am', # Amortization of Intangibles -- didn't look to provide value
              'capxv', # Capital Surplus - kept caps higher-level measure
              'dc', # Deferred Charges - didn't look to provide value
              'diladj', #Dilution Adjustment - most 0 or null
              'do','donr', # Discontinued Operations - most 0 or null
              'emp', # Employees - Don't see the value for this
              'esub', # Equity in Earnings - don't know the value -- over 50% 0 or null
              'exre', # Exchange Rate Effect - don't know the value -- over 50% 0 or null
              'lifr', # LIFO Reserve - don't know the value -- over 50% 0 or null
              'mibt', # Noncontrolling Interests- kept total (mii)
              'prstkc', # Purchase of Common and Preferred Stock - don't know the value -- over 50% 0 or null
              'seqo', # Stakeholder Equity Adjustments -- kept parent (seq)
              'spi', # Special Items - don't know the value -- over 50% 0 or null
              'cshtr_f','dvpsp_f','dvpsx_f','prcc_f','prch_f','prcl_f','adjex_f' # duplicative of Stocks dataset
              ]
fundamentals = fundamentals.drop(columns=colsToDrop)

## Add to report
cleanActionsDesc = 'Third Manual Column Reduce  - Targetted Reduce of Granular Features w/ Total'
df_name = 'Fundamentals'
cols = colsToDrop

cleanActionsReport(fundamentals,df_name,cleanActionsDesc, cols)
cleanActionsDF

,DF,ActionDesc,List-of-Cols,ResultShape
0,Fundamentals,Dropped obvious low value columns -- 1 -- Date...,"[datadate, apdedate, fdate, pdate, fyr, add1, ...","(2323, 1744)"
1,Fundamentals,Initial Null Drop -- columns w/ 63% Null or more.,"[acctchg, acqmeth, adrr, bspr, compst, curuscn...","(2323, 326)"
2,Fundamentals,Dropped columns with only 1 unique value,"[indfmt, consol, popsrc, curcd, final, scf, ac...","(2323, 310)"
3,Fundamentals,Dropping all that are not standard statements ...,"applies to all cols, roughly 50% of rows dropped","(1243, 310)"
4,Fundamentals,Second Null Drop -- columns w/ 20% Null or more.,"[cld2, cld3, cld4, cld5, cshr, dd3, dd4, dd5, ...","(1243, 265)"
5,Fundamentals,Dropping Columns w/ Only 2 values,"[datafmt, ismod, upd, rdip, rdipa, rdipd, rdip...","(1243, 250)"
6,Fundamentals,Second Manual Column Reduce - Dropping Identif...,"[Unnamed: 0, conm, au, auopic, pddur, dpact_fn...","(1243, 240)"
7,Fundamentals,Third Manual Column Reduce - Targetted Reduce...,"[lco, lcox, lcoxdr, lo, loxdr, mib, mibn, ppev...","(1243, 92)"


In [903]:
# Summary Features
## Note to self:  This will cause index matching problems while rinse/repeating through column reduce.  made MD for now.

## Deferred Revenue

fundamentals[['drc','drlt']] = fundamentals[['drc','drlt']].fillna(0)
fundamentals['defrev'] = fundamentals['drc'] + fundamentals['drlt']
fundamentals

## Retained Earnnings Adjustments Revenue

fundamentals[['rea','reajo','recta']] = fundamentals[['rea','reajo','recta']].fillna(0)
fundamentals['defrev'] = fundamentals['rea'] + fundamentals['reajo'] + fundamentals['recta']
fundamentals

## drop source columns once summarized
colsToDrop = ['drc','drlt','rea','reajo','recta']
fundamentals = fundamentals.drop(columns=colsToDrop)

## Add to report
cleanActionsDesc = 'Summary Features  - Used to create features, then dropped'
df_name = 'Fundamentals'
cols = colsToDrop

cleanActionsReport(fundamentals,df_name,cleanActionsDesc, cols)
cleanActionsDF

,DF,ActionDesc,List-of-Cols,ResultShape
0,Fundamentals,Dropped obvious low value columns -- 1 -- Date...,"[datadate, apdedate, fdate, pdate, fyr, add1, ...","(2323, 1744)"
1,Fundamentals,Initial Null Drop -- columns w/ 63% Null or more.,"[acctchg, acqmeth, adrr, bspr, compst, curuscn...","(2323, 326)"
2,Fundamentals,Dropped columns with only 1 unique value,"[indfmt, consol, popsrc, curcd, final, scf, ac...","(2323, 310)"
3,Fundamentals,Dropping all that are not standard statements ...,"applies to all cols, roughly 50% of rows dropped","(1243, 310)"
4,Fundamentals,Second Null Drop -- columns w/ 20% Null or more.,"[cld2, cld3, cld4, cld5, cshr, dd3, dd4, dd5, ...","(1243, 265)"
5,Fundamentals,Dropping Columns w/ Only 2 values,"[datafmt, ismod, upd, rdip, rdipa, rdipd, rdip...","(1243, 250)"
6,Fundamentals,Second Manual Column Reduce - Dropping Identif...,"[Unnamed: 0, conm, au, auopic, pddur, dpact_fn...","(1243, 240)"
7,Fundamentals,Third Manual Column Reduce - Targetted Reduce...,"[lco, lcox, lcoxdr, lo, loxdr, mib, mibn, ppev...","(1243, 92)"
8,Fundamentals,"Summary Features - Used to create features, t...","[drc, drlt, rea, reajo, recta]","(1243, 88)"


### Fundamentals -- Data Dictionary

In [904]:
# Fundamentals Data Dictionary -- Add Column if Still Exists

remainingFeatures = fundamentals.columns.values.tolist()
remainingFeatures= pd.DataFrame(remainingFeatures)
remainingFeatures['Exists'] = 1
remainingFeatures = remainingFeatures.rename(columns={0:'Mnemonic'})

FundamentalsDataDict = fundamentals_DD.merge(remainingFeatures, on='Mnemonic', how='left')
FundamentalsDataDict['Exists'] = FundamentalsDataDict['Exists'].fillna(0)
FundamentalsDataDict

,Mnemonic,Definition,Exists
0,gvkey,Global Company Key,1.0
1,datadate,Data Date,0.0
2,fyear,Data Year - Fiscal,1.0
3,indfmt,Industry Format,0.0
4,consol,Level of Consolidation - Company Annual Descri...,0.0
...,...,...,...
1763,state,State/Province,0.0
1764,stko,Stock Ownership Code,0.0
1765,weburl,Web URL,0.0
1766,dldte,Research Company Deletion Date,0.0


In [905]:
with pd.ExcelWriter(exportPath + 'Fundamentals_DD_wRemaining.xlsx') as writer:
    FundamentalsDataDict.to_excel(writer, sheet_name='Fundamentals_Remaining')

### Fundamentals -- Aggregation

In [906]:
# Create lists iterations

## Create list of numeric categorical cols
fundamentals_numException = ['currtr','src','auop']

## Create List of Cont. Numeric Features
fundamentals_numCols = fundamentals.select_dtypes(include=np.number).columns.tolist()
for col in fundamentals_numException:
        fundamentals_numCols.remove(col)

## Create List of Categorical Features
fundamentals_catCols = fundamentals.select_dtypes(exclude=np.number).columns.tolist()
fundamentals_catCols = fundamentals_catCols + fundamentals_numException +['gvkey']


In [907]:
# make copies of DF to different aggregation logic
fundamentals_numMean = fundamentals[fundamentals_numCols].copy()
fundamentals_numSTD = fundamentals[fundamentals_numCols].copy()
fundamentals_catLast = fundamentals[fundamentals_catCols].copy()

# aggr. each copy
fundamentals_numMean = fundamentals_numMean.groupby('gvkey').mean()
fundamentals_numSTD = fundamentals_numSTD.groupby('gvkey').std().add_suffix(suffix="_std")
fundamentals_catLast = fundamentals_catLast.groupby('gvkey').last()

In [908]:
# Join Aggregration Copies
fundamentals_numAgg = fundamentals_numMean.join(fundamentals_numSTD)
fundamentals_numAgg.columns = sorted(fundamentals_numAgg.columns)
fundamentals_allAgg = fundamentals_catLast.join(fundamentals_numAgg)


# Clean Actions Report
cleanActionsDesc = "Group by GVKey"
df_name = "Fundamentals"
cols = "All"

cleanActionsReport(fundamentals_allAgg,df_name,cleanActionsDesc,cols)
cleanActionsDF

,DF,ActionDesc,List-of-Cols,ResultShape
0,Fundamentals,Dropped obvious low value columns -- 1 -- Date...,"[datadate, apdedate, fdate, pdate, fyr, add1, ...","(2323, 1744)"
1,Fundamentals,Initial Null Drop -- columns w/ 63% Null or more.,"[acctchg, acqmeth, adrr, bspr, compst, curuscn...","(2323, 326)"
2,Fundamentals,Dropped columns with only 1 unique value,"[indfmt, consol, popsrc, curcd, final, scf, ac...","(2323, 310)"
3,Fundamentals,Dropping all that are not standard statements ...,"applies to all cols, roughly 50% of rows dropped","(1243, 310)"
4,Fundamentals,Second Null Drop -- columns w/ 20% Null or more.,"[cld2, cld3, cld4, cld5, cshr, dd3, dd4, dd5, ...","(1243, 265)"
5,Fundamentals,Dropping Columns w/ Only 2 values,"[datafmt, ismod, upd, rdip, rdipa, rdipd, rdip...","(1243, 250)"
6,Fundamentals,Second Manual Column Reduce - Dropping Identif...,"[Unnamed: 0, conm, au, auopic, pddur, dpact_fn...","(1243, 240)"
7,Fundamentals,Third Manual Column Reduce - Targetted Reduce...,"[lco, lcox, lcoxdr, lo, loxdr, mib, mibn, ppev...","(1243, 92)"
8,Fundamentals,"Summary Features - Used to create features, t...","[drc, drlt, rea, reajo, recta]","(1243, 88)"
9,Fundamentals,Group by GVKey,All,"(348, 167)"


In [909]:
# Join Restatement
fund_restatement_summary = fund_restatement_summary.set_index('gvkey')
fundamentals = fundamentals_allAgg.join(fund_restatement_summary)

# Clean Actions Report
cleanActionsDesc = "Join Restatement Data"
df_name = "Fundamentals"
cols = "Restatement Cols"

cleanActionsReport(fundamentals,df_name,cleanActionsDesc,cols)
cleanActionsDF

,DF,ActionDesc,List-of-Cols,ResultShape
0,Fundamentals,Dropped obvious low value columns -- 1 -- Date...,"[datadate, apdedate, fdate, pdate, fyr, add1, ...","(2323, 1744)"
1,Fundamentals,Initial Null Drop -- columns w/ 63% Null or more.,"[acctchg, acqmeth, adrr, bspr, compst, curuscn...","(2323, 326)"
2,Fundamentals,Dropped columns with only 1 unique value,"[indfmt, consol, popsrc, curcd, final, scf, ac...","(2323, 310)"
3,Fundamentals,Dropping all that are not standard statements ...,"applies to all cols, roughly 50% of rows dropped","(1243, 310)"
4,Fundamentals,Second Null Drop -- columns w/ 20% Null or more.,"[cld2, cld3, cld4, cld5, cshr, dd3, dd4, dd5, ...","(1243, 265)"
5,Fundamentals,Dropping Columns w/ Only 2 values,"[datafmt, ismod, upd, rdip, rdipa, rdipd, rdip...","(1243, 250)"
6,Fundamentals,Second Manual Column Reduce - Dropping Identif...,"[Unnamed: 0, conm, au, auopic, pddur, dpact_fn...","(1243, 240)"
7,Fundamentals,Third Manual Column Reduce - Targetted Reduce...,"[lco, lcox, lcoxdr, lo, loxdr, mib, mibn, ppev...","(1243, 92)"
8,Fundamentals,"Summary Features - Used to create features, t...","[drc, drlt, rea, reajo, recta]","(1243, 88)"
9,Fundamentals,Group by GVKey,All,"(348, 167)"


## Ratings

In [910]:
# Seed Clean DF with initial stats
cleanActionsDesc = "Capturing Original Shape"
df_name = "Ratings"
colText = "N/A"

cleanActionsReport(ratings,df_name,cleanActionsDesc,colText)
cleanActionsDF

,DF,ActionDesc,List-of-Cols,ResultShape
0,Fundamentals,Dropped obvious low value columns -- 1 -- Date...,"[datadate, apdedate, fdate, pdate, fyr, add1, ...","(2323, 1744)"
1,Fundamentals,Initial Null Drop -- columns w/ 63% Null or more.,"[acctchg, acqmeth, adrr, bspr, compst, curuscn...","(2323, 326)"
2,Fundamentals,Dropped columns with only 1 unique value,"[indfmt, consol, popsrc, curcd, final, scf, ac...","(2323, 310)"
3,Fundamentals,Dropping all that are not standard statements ...,"applies to all cols, roughly 50% of rows dropped","(1243, 310)"
4,Fundamentals,Second Null Drop -- columns w/ 20% Null or more.,"[cld2, cld3, cld4, cld5, cshr, dd3, dd4, dd5, ...","(1243, 265)"
5,Fundamentals,Dropping Columns w/ Only 2 values,"[datafmt, ismod, upd, rdip, rdipa, rdipd, rdip...","(1243, 250)"
6,Fundamentals,Second Manual Column Reduce - Dropping Identif...,"[Unnamed: 0, conm, au, auopic, pddur, dpact_fn...","(1243, 240)"
7,Fundamentals,Third Manual Column Reduce - Targetted Reduce...,"[lco, lcox, lcoxdr, lo, loxdr, mib, mibn, ppev...","(1243, 92)"
8,Fundamentals,"Summary Features - Used to create features, t...","[drc, drlt, rea, reajo, recta]","(1243, 88)"
9,Fundamentals,Group by GVKey,All,"(348, 167)"


In [911]:
# trim by Gsector -- Only keep my assigned GSector
ratings = ratings[ratings['gsector'] == 30]

cleanActionsDesc = "Trim Rows by GSector == 30"
df_name = "Ratings"
colText = "All"

cleanActionsReport(ratings,df_name,cleanActionsDesc,colText)
cleanActionsDF

,DF,ActionDesc,List-of-Cols,ResultShape
0,Fundamentals,Dropped obvious low value columns -- 1 -- Date...,"[datadate, apdedate, fdate, pdate, fyr, add1, ...","(2323, 1744)"
1,Fundamentals,Initial Null Drop -- columns w/ 63% Null or more.,"[acctchg, acqmeth, adrr, bspr, compst, curuscn...","(2323, 326)"
2,Fundamentals,Dropped columns with only 1 unique value,"[indfmt, consol, popsrc, curcd, final, scf, ac...","(2323, 310)"
3,Fundamentals,Dropping all that are not standard statements ...,"applies to all cols, roughly 50% of rows dropped","(1243, 310)"
4,Fundamentals,Second Null Drop -- columns w/ 20% Null or more.,"[cld2, cld3, cld4, cld5, cshr, dd3, dd4, dd5, ...","(1243, 265)"
5,Fundamentals,Dropping Columns w/ Only 2 values,"[datafmt, ismod, upd, rdip, rdipa, rdipd, rdip...","(1243, 250)"
6,Fundamentals,Second Manual Column Reduce - Dropping Identif...,"[Unnamed: 0, conm, au, auopic, pddur, dpact_fn...","(1243, 240)"
7,Fundamentals,Third Manual Column Reduce - Targetted Reduce...,"[lco, lcox, lcoxdr, lo, loxdr, mib, mibn, ppev...","(1243, 92)"
8,Fundamentals,"Summary Features - Used to create features, t...","[drc, drlt, rea, reajo, recta]","(1243, 88)"
9,Fundamentals,Group by GVKey,All,"(348, 167)"


In [912]:
# Drop Obvious Un-needed 1
## TODO - Note to self -- if a "gcode" is useful, I can drop from fundaments and get here to make that aggr easier.
cleanActionsDesc = "Dropped obvious low value columns -- 1 -- Dates/Codes/Addresses/Etc."
df_name = "Ratings"

colsToDrop = ['city','ggroup','gind','gsector','gsubind','loc','naics',
              'state','conm','tic',
              'idbflag','sic','spcindcd','spcseccd','conml']

ratings = ratings.drop(columns=colsToDrop)
cleanActionsReport(ratings,df_name,cleanActionsDesc,colsToDrop)
cleanActionsDF


,DF,ActionDesc,List-of-Cols,ResultShape
0,Fundamentals,Dropped obvious low value columns -- 1 -- Date...,"[datadate, apdedate, fdate, pdate, fyr, add1, ...","(2323, 1744)"
1,Fundamentals,Initial Null Drop -- columns w/ 63% Null or more.,"[acctchg, acqmeth, adrr, bspr, compst, curuscn...","(2323, 326)"
2,Fundamentals,Dropped columns with only 1 unique value,"[indfmt, consol, popsrc, curcd, final, scf, ac...","(2323, 310)"
3,Fundamentals,Dropping all that are not standard statements ...,"applies to all cols, roughly 50% of rows dropped","(1243, 310)"
4,Fundamentals,Second Null Drop -- columns w/ 20% Null or more.,"[cld2, cld3, cld4, cld5, cshr, dd3, dd4, dd5, ...","(1243, 265)"
5,Fundamentals,Dropping Columns w/ Only 2 values,"[datafmt, ismod, upd, rdip, rdipa, rdipd, rdip...","(1243, 250)"
6,Fundamentals,Second Manual Column Reduce - Dropping Identif...,"[Unnamed: 0, conm, au, auopic, pddur, dpact_fn...","(1243, 240)"
7,Fundamentals,Third Manual Column Reduce - Targetted Reduce...,"[lco, lcox, lcoxdr, lo, loxdr, mib, mibn, ppev...","(1243, 92)"
8,Fundamentals,"Summary Features - Used to create features, t...","[drc, drlt, rea, reajo, recta]","(1243, 88)"
9,Fundamentals,Group by GVKey,All,"(348, 167)"


In [913]:
# Null Analysis Report
ratingsNullReport = nullAnalysis(ratings)
ratingsNullReport

Count of Variables: 6
Count of Variables without nulls: 2
Count of Variables with null : 4


,Variable,Observations,Nulls,Null_Per,Num_Unique,Type
2,spsdrm,17902,17902,100.000,0,float64
3,spsticrm,17902,15476,86.448,5,object
1,splticrm,17902,12272,68.551,19,object
5,spcsrc,17902,8912,49.782,8,object


In [914]:
# Drop Nulls 1
cleanActionsDesc = "Dropped Nulls -- 1 -- More than 50% Null -- Drops more granular Ratings"
df_name = "Ratings"

ratings = dropCols_nullThresh(ratings,0.50,df_name,cleanActionsDesc)
cleanActionsDF

dropped: 3
columns that where dropped where: ['splticrm', 'spsdrm', 'spsticrm']


,DF,ActionDesc,List-of-Cols,ResultShape
0,Fundamentals,Dropped obvious low value columns -- 1 -- Date...,"[datadate, apdedate, fdate, pdate, fyr, add1, ...","(2323, 1744)"
1,Fundamentals,Initial Null Drop -- columns w/ 63% Null or more.,"[acctchg, acqmeth, adrr, bspr, compst, curuscn...","(2323, 326)"
2,Fundamentals,Dropped columns with only 1 unique value,"[indfmt, consol, popsrc, curcd, final, scf, ac...","(2323, 310)"
3,Fundamentals,Dropping all that are not standard statements ...,"applies to all cols, roughly 50% of rows dropped","(1243, 310)"
4,Fundamentals,Second Null Drop -- columns w/ 20% Null or more.,"[cld2, cld3, cld4, cld5, cshr, dd3, dd4, dd5, ...","(1243, 265)"
5,Fundamentals,Dropping Columns w/ Only 2 values,"[datafmt, ismod, upd, rdip, rdipa, rdipd, rdip...","(1243, 250)"
6,Fundamentals,Second Manual Column Reduce - Dropping Identif...,"[Unnamed: 0, conm, au, auopic, pddur, dpact_fn...","(1243, 240)"
7,Fundamentals,Third Manual Column Reduce - Targetted Reduce...,"[lco, lcox, lcoxdr, lo, loxdr, mib, mibn, ppev...","(1243, 92)"
8,Fundamentals,"Summary Features - Used to create features, t...","[drc, drlt, rea, reajo, recta]","(1243, 88)"
9,Fundamentals,Group by GVKey,All,"(348, 167)"


In [915]:
# Oridinal Encode Ratings

## Fill NaN Ratings
ratings['spcsrc'] = ratings['spcsrc'].fillna('NR')

## Set Function Params
ordEncode_Vars = ['spcsrc']
ordEncode_order = ['NR','D','C','B-','B','B+','A-','A','A+']

## Call Ordinal Encoder Func
ratings = OrdinalEncoderFunc (ratings,ordEncode_Vars,ordEncode_order)
ratings

,gvkey,datadate,spcsrc
409,1239,1/31/2010,6.0
410,1239,2/28/2010,6.0
411,1239,3/31/2010,6.0
412,1239,4/30/2010,6.0
413,1239,5/31/2010,6.0
...,...,...,...
123565,277487,6/30/2013,0.0
123566,277487,7/31/2013,0.0
123567,277487,8/31/2013,0.0
123568,277487,9/30/2013,0.0


In [916]:
# Group by
## Note to self -- in a seperate analysis I confirmed that no companies had a change in rating
ratings = ratings.groupby('gvkey')\
    .agg({'datadate':'count','spcsrc':'mean'}).reset_index()

cleanActionsDesc = "Group by GVKey"
df_name = "Ratings"

cleanActionsReport(ratings,df_name,cleanActionsDesc,colsToDrop)
cleanActionsDF

,DF,ActionDesc,List-of-Cols,ResultShape
0,Fundamentals,Dropped obvious low value columns -- 1 -- Date...,"[datadate, apdedate, fdate, pdate, fyr, add1, ...","(2323, 1744)"
1,Fundamentals,Initial Null Drop -- columns w/ 63% Null or more.,"[acctchg, acqmeth, adrr, bspr, compst, curuscn...","(2323, 326)"
2,Fundamentals,Dropped columns with only 1 unique value,"[indfmt, consol, popsrc, curcd, final, scf, ac...","(2323, 310)"
3,Fundamentals,Dropping all that are not standard statements ...,"applies to all cols, roughly 50% of rows dropped","(1243, 310)"
4,Fundamentals,Second Null Drop -- columns w/ 20% Null or more.,"[cld2, cld3, cld4, cld5, cshr, dd3, dd4, dd5, ...","(1243, 265)"
5,Fundamentals,Dropping Columns w/ Only 2 values,"[datafmt, ismod, upd, rdip, rdipa, rdipd, rdip...","(1243, 250)"
6,Fundamentals,Second Manual Column Reduce - Dropping Identif...,"[Unnamed: 0, conm, au, auopic, pddur, dpact_fn...","(1243, 240)"
7,Fundamentals,Third Manual Column Reduce - Targetted Reduce...,"[lco, lcox, lcoxdr, lo, loxdr, mib, mibn, ppev...","(1243, 92)"
8,Fundamentals,"Summary Features - Used to create features, t...","[drc, drlt, rea, reajo, recta]","(1243, 88)"
9,Fundamentals,Group by GVKey,All,"(348, 167)"


## Securities

In [917]:
# Seed Clean DF with initial stats
cleanActionsDesc = "Capturing Original Shape"
df_name = "Securities"
colText = "N/A"

cleanActionsReport(securities,df_name,cleanActionsDesc,colText)
cleanActionsDF


,DF,ActionDesc,List-of-Cols,ResultShape
0,Fundamentals,Dropped obvious low value columns -- 1 -- Date...,"[datadate, apdedate, fdate, pdate, fyr, add1, ...","(2323, 1744)"
1,Fundamentals,Initial Null Drop -- columns w/ 63% Null or more.,"[acctchg, acqmeth, adrr, bspr, compst, curuscn...","(2323, 326)"
2,Fundamentals,Dropped columns with only 1 unique value,"[indfmt, consol, popsrc, curcd, final, scf, ac...","(2323, 310)"
3,Fundamentals,Dropping all that are not standard statements ...,"applies to all cols, roughly 50% of rows dropped","(1243, 310)"
4,Fundamentals,Second Null Drop -- columns w/ 20% Null or more.,"[cld2, cld3, cld4, cld5, cshr, dd3, dd4, dd5, ...","(1243, 265)"
5,Fundamentals,Dropping Columns w/ Only 2 values,"[datafmt, ismod, upd, rdip, rdipa, rdipd, rdip...","(1243, 250)"
6,Fundamentals,Second Manual Column Reduce - Dropping Identif...,"[Unnamed: 0, conm, au, auopic, pddur, dpact_fn...","(1243, 240)"
7,Fundamentals,Third Manual Column Reduce - Targetted Reduce...,"[lco, lcox, lcoxdr, lo, loxdr, mib, mibn, ppev...","(1243, 92)"
8,Fundamentals,"Summary Features - Used to create features, t...","[drc, drlt, rea, reajo, recta]","(1243, 88)"
9,Fundamentals,Group by GVKey,All,"(348, 167)"


In [918]:
# Drop Obvious Un-needed 1
cleanActionsDesc = "Dropped obvious low value columns -- 1 -- Dates/Codes/Addresses/Etc."
df_name = "Securities"

colsToDrop = ['Unnamed: 0','conm','city','conml','cyear','ggroup','gind','gsector','gsubind','loc','naics','sic','state']

securities = securities.drop(columns=colsToDrop)
cleanActionsReport(securities,df_name,cleanActionsDesc,colsToDrop)
cleanActionsDF

,DF,ActionDesc,List-of-Cols,ResultShape
0,Fundamentals,Dropped obvious low value columns -- 1 -- Date...,"[datadate, apdedate, fdate, pdate, fyr, add1, ...","(2323, 1744)"
1,Fundamentals,Initial Null Drop -- columns w/ 63% Null or more.,"[acctchg, acqmeth, adrr, bspr, compst, curuscn...","(2323, 326)"
2,Fundamentals,Dropped columns with only 1 unique value,"[indfmt, consol, popsrc, curcd, final, scf, ac...","(2323, 310)"
3,Fundamentals,Dropping all that are not standard statements ...,"applies to all cols, roughly 50% of rows dropped","(1243, 310)"
4,Fundamentals,Second Null Drop -- columns w/ 20% Null or more.,"[cld2, cld3, cld4, cld5, cshr, dd3, dd4, dd5, ...","(1243, 265)"
5,Fundamentals,Dropping Columns w/ Only 2 values,"[datafmt, ismod, upd, rdip, rdipa, rdipd, rdip...","(1243, 250)"
6,Fundamentals,Second Manual Column Reduce - Dropping Identif...,"[Unnamed: 0, conm, au, auopic, pddur, dpact_fn...","(1243, 240)"
7,Fundamentals,Third Manual Column Reduce - Targetted Reduce...,"[lco, lcox, lcoxdr, lo, loxdr, mib, mibn, ppev...","(1243, 92)"
8,Fundamentals,"Summary Features - Used to create features, t...","[drc, drlt, rea, reajo, recta]","(1243, 88)"
9,Fundamentals,Group by GVKey,All,"(348, 167)"


In [919]:
nullAnalysis_sec = nullAnalysis(securities)
nullAnalysis_sec

Count of Variables: 43
Count of Variables without nulls: 7
Count of Variables with null : 36


,Variable,Observations,Nulls,Null_Per,Num_Unique,Type
19,navm,32382,32382,100.000,0,float64
34,sphvg,32382,32382,100.000,0,float64
32,sphsec,32382,32382,100.000,0,float64
31,sphname,32382,32382,100.000,0,float64
30,sphmid,32382,32382,100.000,0,float64
29,sphiid,32382,32382,100.000,0,float64
28,sphcusip,32382,32382,100.000,0,float64
27,sph100,32382,32382,100.000,0,float64
33,sphtic,32382,32382,100.000,0,float64
11,cheqvm,32382,32363,99.941,19,float64


In [920]:
# Drop Nulls 1
cleanActionsDesc = "Dropped Nulls -- 1 -- More than 20% Null"
df_name = "Securities"

securities = dropCols_nullThresh(securities,0.20,df_name,cleanActionsDesc)
cleanActionsDF

dropped: 23
columns that where dropped where: ['isalrt', 'spgim', 'spiim', 'spmim', 'cheqvm', 'curcddvm', 'dvpspm', 'dvpsxm', 'dvrate', 'csfsm', 'navm', 'rawpm', 'rawxm', 'sph100', 'sphcusip', 'sphiid', 'sphmid', 'sphname', 'sphsec', 'sphtic', 'sphvg', 'cshoq', 'adrrm']


,DF,ActionDesc,List-of-Cols,ResultShape
0,Fundamentals,Dropped obvious low value columns -- 1 -- Date...,"[datadate, apdedate, fdate, pdate, fyr, add1, ...","(2323, 1744)"
1,Fundamentals,Initial Null Drop -- columns w/ 63% Null or more.,"[acctchg, acqmeth, adrr, bspr, compst, curuscn...","(2323, 326)"
2,Fundamentals,Dropped columns with only 1 unique value,"[indfmt, consol, popsrc, curcd, final, scf, ac...","(2323, 310)"
3,Fundamentals,Dropping all that are not standard statements ...,"applies to all cols, roughly 50% of rows dropped","(1243, 310)"
4,Fundamentals,Second Null Drop -- columns w/ 20% Null or more.,"[cld2, cld3, cld4, cld5, cshr, dd3, dd4, dd5, ...","(1243, 265)"
5,Fundamentals,Dropping Columns w/ Only 2 values,"[datafmt, ismod, upd, rdip, rdipa, rdipd, rdip...","(1243, 250)"
6,Fundamentals,Second Manual Column Reduce - Dropping Identif...,"[Unnamed: 0, conm, au, auopic, pddur, dpact_fn...","(1243, 240)"
7,Fundamentals,Third Manual Column Reduce - Targetted Reduce...,"[lco, lcox, lcoxdr, lo, loxdr, mib, mibn, ppev...","(1243, 92)"
8,Fundamentals,"Summary Features - Used to create features, t...","[drc, drlt, rea, reajo, recta]","(1243, 88)"
9,Fundamentals,Group by GVKey,All,"(348, 167)"


In [921]:
# Second Column Reduce -- Only 2 unique Values

lowValCountDF_sec = pd.DataFrame(columns = ['Column','Count','ValueCounts'])
cols = securities.columns.values.tolist()

for col in cols:
    count = securities[col].nunique()
    column = col
    tempVal = securities[col].value_counts(dropna=False)
    tempVal_dict = tempVal.to_dict()
    row = [column,count, tempVal_dict]
    # append stats to Summary Stats DF
    df_length = len(lowValCountDF_sec)
    lowValCountDF_sec.loc[df_length] = row

lowValCountDF_sec = lowValCountDF_sec[lowValCountDF_sec['Count'] <= 4]
lowValCountDF_sec = lowValCountDF_sec.sort_values(by='Count', ascending=True)
lowValCountDF_sec

,Column,Count,ValueCounts
6,primiss,2,"{'P': 30206, 'J': 2127, nan: 49}"
8,curcdm,2,"{'USD': 28317, 'CAD': 3565, nan: 500}"
16,mkvalincl,2,"{'Y': 30534, nan: 1845, 'N': 3}"
19,costat,2,"{'A': 28074, 'I': 4308}"


In [922]:
# Second Column Reduce -- Only 2 unique Values
colsToDrop = lowValCountDF_sec['Column'].values.tolist()
securities = securities.drop(columns=colsToDrop)

cleanActionsDesc = 'Dropping Columns w/ Only 2 values'
df_name = 'Securities'
cols = colsToDrop

cleanActionsReport(securities,df_name,cleanActionsDesc, cols)
cleanActionsDF

,DF,ActionDesc,List-of-Cols,ResultShape
0,Fundamentals,Dropped obvious low value columns -- 1 -- Date...,"[datadate, apdedate, fdate, pdate, fyr, add1, ...","(2323, 1744)"
1,Fundamentals,Initial Null Drop -- columns w/ 63% Null or more.,"[acctchg, acqmeth, adrr, bspr, compst, curuscn...","(2323, 326)"
2,Fundamentals,Dropped columns with only 1 unique value,"[indfmt, consol, popsrc, curcd, final, scf, ac...","(2323, 310)"
3,Fundamentals,Dropping all that are not standard statements ...,"applies to all cols, roughly 50% of rows dropped","(1243, 310)"
4,Fundamentals,Second Null Drop -- columns w/ 20% Null or more.,"[cld2, cld3, cld4, cld5, cshr, dd3, dd4, dd5, ...","(1243, 265)"
5,Fundamentals,Dropping Columns w/ Only 2 values,"[datafmt, ismod, upd, rdip, rdipa, rdipd, rdip...","(1243, 250)"
6,Fundamentals,Second Manual Column Reduce - Dropping Identif...,"[Unnamed: 0, conm, au, auopic, pddur, dpact_fn...","(1243, 240)"
7,Fundamentals,Third Manual Column Reduce - Targetted Reduce...,"[lco, lcox, lcoxdr, lo, loxdr, mib, mibn, ppev...","(1243, 92)"
8,Fundamentals,"Summary Features - Used to create features, t...","[drc, drlt, rea, reajo, recta]","(1243, 88)"
9,Fundamentals,Group by GVKey,All,"(348, 167)"


In [923]:
# Drop Obvious Un-needed 2
cleanActionsDesc = "Manual Drop -- 2 -- Low value or duplicated by other datasets"
df_name = "Securities"

colsToDrop = ['iid','tic','cmth','exchg','tpci', # low value variables
              'cshtrm','prccm','prchm','prclm','cshom' # redundant to variables from other datasets
              ]

securities = securities.drop(columns=colsToDrop)
cleanActionsReport(securities,df_name,cleanActionsDesc,colsToDrop)
cleanActionsDF

,DF,ActionDesc,List-of-Cols,ResultShape
0,Fundamentals,Dropped obvious low value columns -- 1 -- Date...,"[datadate, apdedate, fdate, pdate, fyr, add1, ...","(2323, 1744)"
1,Fundamentals,Initial Null Drop -- columns w/ 63% Null or more.,"[acctchg, acqmeth, adrr, bspr, compst, curuscn...","(2323, 326)"
2,Fundamentals,Dropped columns with only 1 unique value,"[indfmt, consol, popsrc, curcd, final, scf, ac...","(2323, 310)"
3,Fundamentals,Dropping all that are not standard statements ...,"applies to all cols, roughly 50% of rows dropped","(1243, 310)"
4,Fundamentals,Second Null Drop -- columns w/ 20% Null or more.,"[cld2, cld3, cld4, cld5, cshr, dd3, dd4, dd5, ...","(1243, 265)"
5,Fundamentals,Dropping Columns w/ Only 2 values,"[datafmt, ismod, upd, rdip, rdipa, rdipd, rdip...","(1243, 250)"
6,Fundamentals,Second Manual Column Reduce - Dropping Identif...,"[Unnamed: 0, conm, au, auopic, pddur, dpact_fn...","(1243, 240)"
7,Fundamentals,Third Manual Column Reduce - Targetted Reduce...,"[lco, lcox, lcoxdr, lo, loxdr, mib, mibn, ppev...","(1243, 92)"
8,Fundamentals,"Summary Features - Used to create features, t...","[drc, drlt, rea, reajo, recta]","(1243, 88)"
9,Fundamentals,Group by GVKey,All,"(348, 167)"


In [924]:
# Group By Prep
## Insert STD cols
securities.insert(5,"trfm_std", 0)
securities['trfm_std'] = securities['trfm']

securities.insert(7,"trt1m_std", 0)
securities['trt1m_std'] = securities['trt1m']

## Rename columns
securities = securities.rename(columns={'trfm':'trfm_mean','trt1m':'trt1m_mean'})

In [925]:
# Group by GVKey
securities = securities.groupby('gvkey')\
    .agg({'ajexm':'mean','ajpm':'mean',
          'trfm_mean':'mean','trfm_std':'std',
          'trt1m_mean':'mean','trt1m_std':'std'}).reset_index()

# Call Clean Report Function
cleanActionsDesc = "Group by GVKey"
df_name = "Securities"

cleanActionsReport(securities,df_name,cleanActionsDesc,colsToDrop)
cleanActionsDF

,DF,ActionDesc,List-of-Cols,ResultShape
0,Fundamentals,Dropped obvious low value columns -- 1 -- Date...,"[datadate, apdedate, fdate, pdate, fyr, add1, ...","(2323, 1744)"
1,Fundamentals,Initial Null Drop -- columns w/ 63% Null or more.,"[acctchg, acqmeth, adrr, bspr, compst, curuscn...","(2323, 326)"
2,Fundamentals,Dropped columns with only 1 unique value,"[indfmt, consol, popsrc, curcd, final, scf, ac...","(2323, 310)"
3,Fundamentals,Dropping all that are not standard statements ...,"applies to all cols, roughly 50% of rows dropped","(1243, 310)"
4,Fundamentals,Second Null Drop -- columns w/ 20% Null or more.,"[cld2, cld3, cld4, cld5, cshr, dd3, dd4, dd5, ...","(1243, 265)"
5,Fundamentals,Dropping Columns w/ Only 2 values,"[datafmt, ismod, upd, rdip, rdipa, rdipd, rdip...","(1243, 250)"
6,Fundamentals,Second Manual Column Reduce - Dropping Identif...,"[Unnamed: 0, conm, au, auopic, pddur, dpact_fn...","(1243, 240)"
7,Fundamentals,Third Manual Column Reduce - Targetted Reduce...,"[lco, lcox, lcoxdr, lo, loxdr, mib, mibn, ppev...","(1243, 92)"
8,Fundamentals,"Summary Features - Used to create features, t...","[drc, drlt, rea, reajo, recta]","(1243, 88)"
9,Fundamentals,Group by GVKey,All,"(348, 167)"


### Securities -- Data Dictionary

In [926]:
# Securities Data Dictionary -- Add Column if Still Exists

remainingFeatures_sec = securities.columns.values.tolist()
remainingFeatures_sec= pd.DataFrame(remainingFeatures_sec)
remainingFeatures_sec['Exists'] = 1
remainingFeatures_sec = remainingFeatures_sec.rename(columns={0:'Mnemonic'})

securitiesDataDict = securities_DD.merge(remainingFeatures_sec, on='Mnemonic', how='left')
securitiesDataDict['Exists'] = securitiesDataDict['Exists'].fillna(0)
securitiesDataDict

,Mnemonic,Definition,Exists
0,gvkey,Global Company Key,1.0
1,iid,Issue ID - Security Monthly Descriptor,0.0
2,datadate,Data Date - Security Monthly,0.0
3,tic,Ticker Symbol,0.0
4,conm,Company Name,0.0
5,ajexm,Cumulative Adjustment Factor - Ex Date -Monthly,1.0
6,ajpm,Cumulative Adjustment Factor - Pay Date -Monthly,1.0
7,isalrt,Status Code,0.0
8,primiss,Primary/Joiner flag,0.0
9,spgim,S&P GICS Index Code - Historical,0.0


In [927]:
with pd.ExcelWriter(exportPath + 'Securities_DD_wRemaining.xlsx') as writer:
    securitiesDataDict.to_excel(writer, sheet_name='Securities_Remaining')

## Stocks

In [928]:
# Seed Clean DF with initial stats
cleanActionsDesc = "Capturing Original Shape"
df_name = "Stocks"
colText = "N/A"

cleanActionsReport(stocks,df_name,cleanActionsDesc,colText)
cleanActionsDF

,DF,ActionDesc,List-of-Cols,ResultShape
0,Fundamentals,Dropped obvious low value columns -- 1 -- Date...,"[datadate, apdedate, fdate, pdate, fyr, add1, ...","(2323, 1744)"
1,Fundamentals,Initial Null Drop -- columns w/ 63% Null or more.,"[acctchg, acqmeth, adrr, bspr, compst, curuscn...","(2323, 326)"
2,Fundamentals,Dropped columns with only 1 unique value,"[indfmt, consol, popsrc, curcd, final, scf, ac...","(2323, 310)"
3,Fundamentals,Dropping all that are not standard statements ...,"applies to all cols, roughly 50% of rows dropped","(1243, 310)"
4,Fundamentals,Second Null Drop -- columns w/ 20% Null or more.,"[cld2, cld3, cld4, cld5, cshr, dd3, dd4, dd5, ...","(1243, 265)"
5,Fundamentals,Dropping Columns w/ Only 2 values,"[datafmt, ismod, upd, rdip, rdipa, rdipd, rdip...","(1243, 250)"
6,Fundamentals,Second Manual Column Reduce - Dropping Identif...,"[Unnamed: 0, conm, au, auopic, pddur, dpact_fn...","(1243, 240)"
7,Fundamentals,Third Manual Column Reduce - Targetted Reduce...,"[lco, lcox, lcoxdr, lo, loxdr, mib, mibn, ppev...","(1243, 92)"
8,Fundamentals,"Summary Features - Used to create features, t...","[drc, drlt, rea, reajo, recta]","(1243, 88)"
9,Fundamentals,Group by GVKey,All,"(348, 167)"


In [929]:
nullAnalysis_stocks = nullAnalysis(stocks)
nullAnalysis_stocks

Count of Variables: 77
Count of Variables without nulls: 24
Count of Variables with null : 53


,Variable,Observations,Nulls,Null_Per,Num_Unique,Type
17,capgnpaydate,673564,673564,100.000,0,float64
8,capgn,673564,673564,100.000,0,float64
9,cheqv,673564,673545,99.997,19,float64
18,cheqvpaydate,673564,673545,99.997,16,object
12,divdpaydateind,673564,673539,99.996,1,object
15,paydateind,673564,673537,99.996,1,object
20,divsppaydate,673564,673427,99.980,100,object
13,divsp,673564,673427,99.980,91,float64
51,county,673564,672216,99.800,3,object
16,anncdate,673564,670738,99.580,882,object


In [930]:
# Drop Nulls 1
cleanActionsDesc = "Dropped Nulls -- 1 -- More than 20% Null"
df_name = "Stocks"

stocks = dropCols_nullThresh(stocks,0.20,df_name,cleanActionsDesc)


dropped: 37
columns that where dropped where: ['curcddv', 'capgn', 'cheqv', 'div', 'divd', 'divdpaydateind', 'divsp', 'dvrated', 'paydateind', 'anncdate', 'capgnpaydate', 'cheqvpaydate', 'divdpaydate', 'divsppaydate', 'paydate', 'recorddate', 'adrrc', 'dvi', 'eps', 'epsmo', 'prcod', 'add2', 'add3', 'add4', 'county', 'dlrsn', 'ein', 'fax', 'incorp', 'prican', 'prirow', 'spcindcd', 'spcseccd', 'spcsrc', 'state', 'dldte', 'ipodate']


In [931]:
# Drop Obvious Un-needed 1
cleanActionsDesc = "Dropped obvious low value columns -- 1 -- Dates/Codes/Addresses/Etc."
df_name = "Stocks"

colsToDrop = ['Unnamed: 0','tic','conm','curcdd','fic','add1','addzip','busdesc','city','conml','fyrc','ggroup','gind',
              'gsector','gsubind','idbflag','loc','naics','phone','sic','weburl','prcstd','cusip','priusa']

stocks = stocks.drop(columns=colsToDrop)
cleanActionsReport(stocks,df_name,cleanActionsDesc,colsToDrop)


,DF,ActionDesc,List-of-Cols,ResultShape
0,Fundamentals,Dropped obvious low value columns -- 1 -- Date...,"[datadate, apdedate, fdate, pdate, fyr, add1, ...","(2323, 1744)"
1,Fundamentals,Initial Null Drop -- columns w/ 63% Null or more.,"[acctchg, acqmeth, adrr, bspr, compst, curuscn...","(2323, 326)"
2,Fundamentals,Dropped columns with only 1 unique value,"[indfmt, consol, popsrc, curcd, final, scf, ac...","(2323, 310)"
3,Fundamentals,Dropping all that are not standard statements ...,"applies to all cols, roughly 50% of rows dropped","(1243, 310)"
4,Fundamentals,Second Null Drop -- columns w/ 20% Null or more.,"[cld2, cld3, cld4, cld5, cshr, dd3, dd4, dd5, ...","(1243, 265)"
5,Fundamentals,Dropping Columns w/ Only 2 values,"[datafmt, ismod, upd, rdip, rdipa, rdipd, rdip...","(1243, 250)"
6,Fundamentals,Second Manual Column Reduce - Dropping Identif...,"[Unnamed: 0, conm, au, auopic, pddur, dpact_fn...","(1243, 240)"
7,Fundamentals,Third Manual Column Reduce - Targetted Reduce...,"[lco, lcox, lcoxdr, lo, loxdr, mib, mibn, ppev...","(1243, 92)"
8,Fundamentals,"Summary Features - Used to create features, t...","[drc, drlt, rea, reajo, recta]","(1243, 88)"
9,Fundamentals,Group by GVKey,All,"(348, 167)"


In [932]:
# Second Column Reduce -- Only X unique Values

lowValCountDF_stocks = pd.DataFrame(columns = ['Column','Count','ValueCounts'])
cols = stocks.columns.values.tolist()

for col in cols:
    count = stocks[col].nunique()
    column = col
    tempVal = stocks[col].value_counts(dropna=False)
    tempVal_dict = tempVal.to_dict()
    row = [column,count, tempVal_dict]
    # append stats to Summary Stats DF
    df_length = len(lowValCountDF_stocks)
    lowValCountDF_stocks.loc[df_length] = row

lowValCountDF_stocks = lowValCountDF_stocks[lowValCountDF_stocks['Count'] <= 4]
lowValCountDF_stocks = lowValCountDF_stocks.sort_values(by='Count', ascending=True)
lowValCountDF_stocks

,Column,Count,ValueCounts
11,secstat,2,"{'A': 588583, 'I': 84981}"
14,costat,2,"{'A': 583441, 'I': 90123}"
15,stko,4,"{0.0: 341046, 3.0: 328872, 2.0: 1427, nan: 117..."


In [933]:
# Second Column Reduce -- Only 2 unique Values
colsToDrop = lowValCountDF_stocks['Column'].values.tolist()
stocks = stocks.drop(columns=colsToDrop)

cleanActionsDesc = 'Dropping Columns w/ Only 2 values'
df_name = 'Stocks'
cols = colsToDrop

cleanActionsReport(stocks,df_name,cleanActionsDesc, cols)

,DF,ActionDesc,List-of-Cols,ResultShape
0,Fundamentals,Dropped obvious low value columns -- 1 -- Date...,"[datadate, apdedate, fdate, pdate, fyr, add1, ...","(2323, 1744)"
1,Fundamentals,Initial Null Drop -- columns w/ 63% Null or more.,"[acctchg, acqmeth, adrr, bspr, compst, curuscn...","(2323, 326)"
2,Fundamentals,Dropped columns with only 1 unique value,"[indfmt, consol, popsrc, curcd, final, scf, ac...","(2323, 310)"
3,Fundamentals,Dropping all that are not standard statements ...,"applies to all cols, roughly 50% of rows dropped","(1243, 310)"
4,Fundamentals,Second Null Drop -- columns w/ 20% Null or more.,"[cld2, cld3, cld4, cld5, cshr, dd3, dd4, dd5, ...","(1243, 265)"
5,Fundamentals,Dropping Columns w/ Only 2 values,"[datafmt, ismod, upd, rdip, rdipa, rdipd, rdip...","(1243, 250)"
6,Fundamentals,Second Manual Column Reduce - Dropping Identif...,"[Unnamed: 0, conm, au, auopic, pddur, dpact_fn...","(1243, 240)"
7,Fundamentals,Third Manual Column Reduce - Targetted Reduce...,"[lco, lcox, lcoxdr, lo, loxdr, mib, mibn, ppev...","(1243, 92)"
8,Fundamentals,"Summary Features - Used to create features, t...","[drc, drlt, rea, reajo, recta]","(1243, 88)"
9,Fundamentals,Group by GVKey,All,"(348, 167)"


In [934]:
# Drop Obvious Un-needed 2
cleanActionsDesc = "Manual Drop -- 2 -- Low value or duplicated by other datasets"
df_name = "Stocks"

colsToDrop = ['iid','exchg','tpci','cik','cshtrd', # low value variables
              'ajexdi','trfd' # Duplicative of Securities Dataset
              ]

stocks = stocks.drop(columns=colsToDrop)
cleanActionsReport(stocks,df_name,cleanActionsDesc,colsToDrop)


,DF,ActionDesc,List-of-Cols,ResultShape
0,Fundamentals,Dropped obvious low value columns -- 1 -- Date...,"[datadate, apdedate, fdate, pdate, fyr, add1, ...","(2323, 1744)"
1,Fundamentals,Initial Null Drop -- columns w/ 63% Null or more.,"[acctchg, acqmeth, adrr, bspr, compst, curuscn...","(2323, 326)"
2,Fundamentals,Dropped columns with only 1 unique value,"[indfmt, consol, popsrc, curcd, final, scf, ac...","(2323, 310)"
3,Fundamentals,Dropping all that are not standard statements ...,"applies to all cols, roughly 50% of rows dropped","(1243, 310)"
4,Fundamentals,Second Null Drop -- columns w/ 20% Null or more.,"[cld2, cld3, cld4, cld5, cshr, dd3, dd4, dd5, ...","(1243, 265)"
5,Fundamentals,Dropping Columns w/ Only 2 values,"[datafmt, ismod, upd, rdip, rdipa, rdipd, rdip...","(1243, 250)"
6,Fundamentals,Second Manual Column Reduce - Dropping Identif...,"[Unnamed: 0, conm, au, auopic, pddur, dpact_fn...","(1243, 240)"
7,Fundamentals,Third Manual Column Reduce - Targetted Reduce...,"[lco, lcox, lcoxdr, lo, loxdr, mib, mibn, ppev...","(1243, 92)"
8,Fundamentals,"Summary Features - Used to create features, t...","[drc, drlt, rea, reajo, recta]","(1243, 88)"
9,Fundamentals,Group by GVKey,All,"(348, 167)"


In [935]:
# Group By Prep
## Insert STD cols
stocks.insert(3,"cshoc_std", 0)
stocks['cshoc_std'] = stocks['cshoc']

stocks.insert(6,"prccd_std", 0)
stocks['prccd_std'] = stocks['prccd']

stocks.insert(7,"prc_start", 0)
stocks['prc_start'] = stocks['prccd']

stocks.insert(8,"prc_end", 0)
stocks['prc_end'] = stocks['prccd']

## Rename columns
stocks = stocks.rename(columns={'cshoc':'cshoc_med',
                                'prchd':'prc_at_high','prcld':'prc_at_low',
                                'prccd':'prccd_mean'})

In [936]:
# Group by GVKey
stocks = stocks.groupby('gvkey')\
    .agg({'cshoc_med':'median','cshoc_std':'std',
          'prccd_mean':'mean','prccd_std':'std',
          'prc_start':'first','prc_end':'last',
          'prc_at_high':'max','prc_at_low':'min'}).reset_index()

# Call Clean Report Function
cleanActionsDesc = "Group by GVKey"
df_name = "Stocks"

cleanActionsReport(stocks,df_name,cleanActionsDesc,colsToDrop)

,DF,ActionDesc,List-of-Cols,ResultShape
0,Fundamentals,Dropped obvious low value columns -- 1 -- Date...,"[datadate, apdedate, fdate, pdate, fyr, add1, ...","(2323, 1744)"
1,Fundamentals,Initial Null Drop -- columns w/ 63% Null or more.,"[acctchg, acqmeth, adrr, bspr, compst, curuscn...","(2323, 326)"
2,Fundamentals,Dropped columns with only 1 unique value,"[indfmt, consol, popsrc, curcd, final, scf, ac...","(2323, 310)"
3,Fundamentals,Dropping all that are not standard statements ...,"applies to all cols, roughly 50% of rows dropped","(1243, 310)"
4,Fundamentals,Second Null Drop -- columns w/ 20% Null or more.,"[cld2, cld3, cld4, cld5, cshr, dd3, dd4, dd5, ...","(1243, 265)"
5,Fundamentals,Dropping Columns w/ Only 2 values,"[datafmt, ismod, upd, rdip, rdipa, rdipd, rdip...","(1243, 250)"
6,Fundamentals,Second Manual Column Reduce - Dropping Identif...,"[Unnamed: 0, conm, au, auopic, pddur, dpact_fn...","(1243, 240)"
7,Fundamentals,Third Manual Column Reduce - Targetted Reduce...,"[lco, lcox, lcoxdr, lo, loxdr, mib, mibn, ppev...","(1243, 92)"
8,Fundamentals,"Summary Features - Used to create features, t...","[drc, drlt, rea, reajo, recta]","(1243, 88)"
9,Fundamentals,Group by GVKey,All,"(348, 167)"


In [937]:
# Round Variables
colsToRound = ['cshoc_med','cshoc_std','prccd_mean','prccd_std','prc_start','prc_end',
               'prc_at_high','prc_at_low']

for col in colsToRound:
    stocks[col] = round(stocks[col],2)


In [938]:
# reduce to a few calculated Fields

## 1 -- percent growth from start to end

stocks['diff'] = stocks['prc_end'] - stocks['prc_start']
stocks['per_growth'] = round(stocks['diff'] / stocks['prc_start'],2)

## 2 -- percent of most recent closing to max
stocks['per_currentToMax'] = round(stocks['prc_end'] / stocks['prc_at_high'],2)

## 3  -- percent of low to starting
stocks['per_lowToStart'] = round((stocks['prc_at_low'] / stocks['prc_start']-1),2)

## 4 -- Volatility: std as % of mean
stocks['volatility'] = round(stocks['prccd_std'] / stocks['prccd_mean'],2)

## drop other price vars no longer needed
stocks = stocks.drop(columns=['prccd_mean','prccd_std','prc_start','prc_at_high','prc_at_low','diff'])

# Call Clean Report Function
cleanActionsDesc = "Add Calculated Fields -- replace variables no longer needed"
df_name = "Stocks"

cleanActionsReport(stocks,df_name,cleanActionsDesc,colsToDrop)

,DF,ActionDesc,List-of-Cols,ResultShape
0,Fundamentals,Dropped obvious low value columns -- 1 -- Date...,"[datadate, apdedate, fdate, pdate, fyr, add1, ...","(2323, 1744)"
1,Fundamentals,Initial Null Drop -- columns w/ 63% Null or more.,"[acctchg, acqmeth, adrr, bspr, compst, curuscn...","(2323, 326)"
2,Fundamentals,Dropped columns with only 1 unique value,"[indfmt, consol, popsrc, curcd, final, scf, ac...","(2323, 310)"
3,Fundamentals,Dropping all that are not standard statements ...,"applies to all cols, roughly 50% of rows dropped","(1243, 310)"
4,Fundamentals,Second Null Drop -- columns w/ 20% Null or more.,"[cld2, cld3, cld4, cld5, cshr, dd3, dd4, dd5, ...","(1243, 265)"
5,Fundamentals,Dropping Columns w/ Only 2 values,"[datafmt, ismod, upd, rdip, rdipa, rdipd, rdip...","(1243, 250)"
6,Fundamentals,Second Manual Column Reduce - Dropping Identif...,"[Unnamed: 0, conm, au, auopic, pddur, dpact_fn...","(1243, 240)"
7,Fundamentals,Third Manual Column Reduce - Targetted Reduce...,"[lco, lcox, lcoxdr, lo, loxdr, mib, mibn, ppev...","(1243, 92)"
8,Fundamentals,"Summary Features - Used to create features, t...","[drc, drlt, rea, reajo, recta]","(1243, 88)"
9,Fundamentals,Group by GVKey,All,"(348, 167)"


### Stocks -- Data Dictionary

In [939]:
# Stocks Data Dictionary -- Add Column if Still Exists

remainingFeatures_stocks = stocks.columns.values.tolist()
remainingFeatures_stocks= pd.DataFrame(remainingFeatures_stocks)
remainingFeatures_stocks['Exists'] = 1
remainingFeatures_stocks = remainingFeatures_stocks.rename(columns={0:'Mnemonic'})

stocksDataDict = stocks_DD.merge(remainingFeatures_stocks, on='Mnemonic', how='left')
stocksDataDict['Exists'] = stocksDataDict['Exists'].fillna(0)
stocksDataDict

,Mnemonic,Definition,Exists
0,gvkey,Global Company Key,1.0
1,iid,Issue ID - Dividends,0.0
2,datadate,Data Date - Dividends,0.0
3,tic,Ticker Symbol,0.0
4,cusip,CUSIP,0.0
...,...,...,...
71,state,State/Province,0.0
72,stko,Stock Ownership Code,0.0
73,weburl,Web URL,0.0
74,dldte,Research Company Deletion Date,0.0


In [940]:
with pd.ExcelWriter(exportPath + 'Stocks_DD_wRemaining.xlsx') as writer:
    stocksDataDict.to_excel(writer, sheet_name='Stocks_Remaining')


## SCA Filings

In [941]:
# Seed Clean DF with initial stats
cleanActionsDesc = "Capturing Original Shape"
df_name = "SCA_Filings"
colText = "N/A"

cleanActionsReport(sca_filings,df_name,cleanActionsDesc,colText)
cleanActionsDF

,DF,ActionDesc,List-of-Cols,ResultShape
0,Fundamentals,Dropped obvious low value columns -- 1 -- Date...,"[datadate, apdedate, fdate, pdate, fyr, add1, ...","(2323, 1744)"
1,Fundamentals,Initial Null Drop -- columns w/ 63% Null or more.,"[acctchg, acqmeth, adrr, bspr, compst, curuscn...","(2323, 326)"
2,Fundamentals,Dropped columns with only 1 unique value,"[indfmt, consol, popsrc, curcd, final, scf, ac...","(2323, 310)"
3,Fundamentals,Dropping all that are not standard statements ...,"applies to all cols, roughly 50% of rows dropped","(1243, 310)"
4,Fundamentals,Second Null Drop -- columns w/ 20% Null or more.,"[cld2, cld3, cld4, cld5, cshr, dd3, dd4, dd5, ...","(1243, 265)"
5,Fundamentals,Dropping Columns w/ Only 2 values,"[datafmt, ismod, upd, rdip, rdipa, rdipd, rdip...","(1243, 250)"
6,Fundamentals,Second Manual Column Reduce - Dropping Identif...,"[Unnamed: 0, conm, au, auopic, pddur, dpact_fn...","(1243, 240)"
7,Fundamentals,Third Manual Column Reduce - Targetted Reduce...,"[lco, lcox, lcoxdr, lo, loxdr, mib, mibn, ppev...","(1243, 92)"
8,Fundamentals,"Summary Features - Used to create features, t...","[drc, drlt, rea, reajo, recta]","(1243, 88)"
9,Fundamentals,Group by GVKey,All,"(348, 167)"


In [942]:
sca_filings

,FilingName,Exchange,Ticker,Dismissed,SettlementAmount,FilingYear
0,"AAC Holdings, Inc.",New York SE,AAC,Ongoing,#NULL!,2015
1,"Airtran Holdings, Inc.",New York SE,AAI,Yes,#NULL!,2011
2,Altisource Asset Management Corporation,New York SE,AAMC,Ongoing,#NULL!,2015
3,"APPLE Computer, Inc.",NASDAQ,AAPL,No,16500000,2006
4,"Avalanche Biotechnologies, Inc.",NASDAQ,AAVL,Ongoing,#NULL!,2015
...,...,...,...,...,...,...
1887,"Quiksilver, Inc.",New York SE,ZQK,Ongoing,#NULL!,2015
1888,Zoran Corporation,NASDAQ,ZRAN,Yes,#NULL!,2006
1889,"ZST Digital Networks, Inc.",NASDAQ,ZSTN,No,1700000,2011
1890,"Zumiez, Inc.",NASDAQ,ZUMZ,Yes,#NULL!,2007


In [943]:
# Drop Unneeded Columns

cleanActionsDesc = "Manual Drop -- 1 -- Low value or duplicated by other datasets"
df_name = "SCA_Filings"

colsToDrop = ['FilingName','Exchange','FilingYear','Dismissed']

sca_filings = sca_filings.drop(columns=colsToDrop)
cleanActionsReport(sca_filings,df_name,cleanActionsDesc,colsToDrop)

,DF,ActionDesc,List-of-Cols,ResultShape
0,Fundamentals,Dropped obvious low value columns -- 1 -- Date...,"[datadate, apdedate, fdate, pdate, fyr, add1, ...","(2323, 1744)"
1,Fundamentals,Initial Null Drop -- columns w/ 63% Null or more.,"[acctchg, acqmeth, adrr, bspr, compst, curuscn...","(2323, 326)"
2,Fundamentals,Dropped columns with only 1 unique value,"[indfmt, consol, popsrc, curcd, final, scf, ac...","(2323, 310)"
3,Fundamentals,Dropping all that are not standard statements ...,"applies to all cols, roughly 50% of rows dropped","(1243, 310)"
4,Fundamentals,Second Null Drop -- columns w/ 20% Null or more.,"[cld2, cld3, cld4, cld5, cshr, dd3, dd4, dd5, ...","(1243, 265)"
5,Fundamentals,Dropping Columns w/ Only 2 values,"[datafmt, ismod, upd, rdip, rdipa, rdipd, rdip...","(1243, 250)"
6,Fundamentals,Second Manual Column Reduce - Dropping Identif...,"[Unnamed: 0, conm, au, auopic, pddur, dpact_fn...","(1243, 240)"
7,Fundamentals,Third Manual Column Reduce - Targetted Reduce...,"[lco, lcox, lcoxdr, lo, loxdr, mib, mibn, ppev...","(1243, 92)"
8,Fundamentals,"Summary Features - Used to create features, t...","[drc, drlt, rea, reajo, recta]","(1243, 88)"
9,Fundamentals,Group by GVKey,All,"(348, 167)"


In [944]:
## TODO: to agg -- need to fill nulls w/0 which was against the professor's requirements.
###  Either back this out depending on answer from professor OR convert 0's back to null.
sca_filings['SettlementAmount'] = sca_filings['SettlementAmount'].replace({"#NULL!":0})
sca_filings['SettlementAmount'].value_counts()

0.0            1433
10000000.0       14
3000000.0        10
2000000.0        10
5000000.0         9
               ... 
15250000.0        1
26612500.0        1
22900000.0        1
13600000.0        1
500000000.0       1
Name: SettlementAmount, Length: 241, dtype: int64

In [945]:
# Group by Ticker

## TODO: Open question on whether I should aggregrate to a single row.

sca_filings = sca_filings.groupby('Ticker')\
    .agg({'SettlementAmount':'max'}).reset_index()

# Call Clean Report Function
cleanActionsDesc = "Group by Ticker"
df_name = "SCA_Filings"
cols = "All"

cleanActionsReport(sca_filings,df_name,cleanActionsDesc,colsToDrop)

,DF,ActionDesc,List-of-Cols,ResultShape
0,Fundamentals,Dropped obvious low value columns -- 1 -- Date...,"[datadate, apdedate, fdate, pdate, fyr, add1, ...","(2323, 1744)"
1,Fundamentals,Initial Null Drop -- columns w/ 63% Null or more.,"[acctchg, acqmeth, adrr, bspr, compst, curuscn...","(2323, 326)"
2,Fundamentals,Dropped columns with only 1 unique value,"[indfmt, consol, popsrc, curcd, final, scf, ac...","(2323, 310)"
3,Fundamentals,Dropping all that are not standard statements ...,"applies to all cols, roughly 50% of rows dropped","(1243, 310)"
4,Fundamentals,Second Null Drop -- columns w/ 20% Null or more.,"[cld2, cld3, cld4, cld5, cshr, dd3, dd4, dd5, ...","(1243, 265)"
5,Fundamentals,Dropping Columns w/ Only 2 values,"[datafmt, ismod, upd, rdip, rdipa, rdipd, rdip...","(1243, 250)"
6,Fundamentals,Second Manual Column Reduce - Dropping Identif...,"[Unnamed: 0, conm, au, auopic, pddur, dpact_fn...","(1243, 240)"
7,Fundamentals,Third Manual Column Reduce - Targetted Reduce...,"[lco, lcox, lcoxdr, lo, loxdr, mib, mibn, ppev...","(1243, 92)"
8,Fundamentals,"Summary Features - Used to create features, t...","[drc, drlt, rea, reajo, recta]","(1243, 88)"
9,Fundamentals,Group by GVKey,All,"(348, 167)"


In [946]:
sca_filings

,Ticker,SettlementAmount
0,AAC,0.0
1,AAI,0.0
2,AAMC,0.0
3,AAPL,16500000.0
4,AAVL,0.0
...,...,...
1501,ZQK,0.0
1502,ZRAN,0.0
1503,ZSTN,1700000.0
1504,ZUMZ,0.0


In [947]:
# Create Dummy Vars into a single Response Var
sca_filings.insert(1,'lawsuit', 1)

cleanActionsDesc = "Create Boolean Response Var"
df_name = "SCA_Filings"
cleanActionsReport(sca_filings,df_name,cleanActionsDesc,colsToDrop)
sca_filings

,Ticker,lawsuit,SettlementAmount
0,AAC,1,0.0
1,AAI,1,0.0
2,AAMC,1,0.0
3,AAPL,1,16500000.0
4,AAVL,1,0.0
...,...,...,...
1501,ZQK,1,0.0
1502,ZRAN,1,0.0
1503,ZSTN,1,1700000.0
1504,ZUMZ,1,0.0


In [948]:
sca_filings = sca_filings.rename(columns={'Ticker':'tic'})
sca_filings

,tic,lawsuit,SettlementAmount
0,AAC,1,0.0
1,AAI,1,0.0
2,AAMC,1,0.0
3,AAPL,1,16500000.0
4,AAVL,1,0.0
...,...,...,...
1501,ZQK,1,0.0
1502,ZRAN,1,0.0
1503,ZSTN,1,1700000.0
1504,ZUMZ,1,0.0


In [949]:
# Note to self -- 0's or abnormal ammounts not in my overall dataset
sca_filings['lawsuit'].value_counts(dropna=False)

1    1506
Name: lawsuit, dtype: int64

In [950]:
cleanActionsDF

,DF,ActionDesc,List-of-Cols,ResultShape
0,Fundamentals,Dropped obvious low value columns -- 1 -- Date...,"[datadate, apdedate, fdate, pdate, fyr, add1, ...","(2323, 1744)"
1,Fundamentals,Initial Null Drop -- columns w/ 63% Null or more.,"[acctchg, acqmeth, adrr, bspr, compst, curuscn...","(2323, 326)"
2,Fundamentals,Dropped columns with only 1 unique value,"[indfmt, consol, popsrc, curcd, final, scf, ac...","(2323, 310)"
3,Fundamentals,Dropping all that are not standard statements ...,"applies to all cols, roughly 50% of rows dropped","(1243, 310)"
4,Fundamentals,Second Null Drop -- columns w/ 20% Null or more.,"[cld2, cld3, cld4, cld5, cshr, dd3, dd4, dd5, ...","(1243, 265)"
5,Fundamentals,Dropping Columns w/ Only 2 values,"[datafmt, ismod, upd, rdip, rdipa, rdipd, rdip...","(1243, 250)"
6,Fundamentals,Second Manual Column Reduce - Dropping Identif...,"[Unnamed: 0, conm, au, auopic, pddur, dpact_fn...","(1243, 240)"
7,Fundamentals,Third Manual Column Reduce - Targetted Reduce...,"[lco, lcox, lcoxdr, lo, loxdr, mib, mibn, ppev...","(1243, 92)"
8,Fundamentals,"Summary Features - Used to create features, t...","[drc, drlt, rea, reajo, recta]","(1243, 88)"
9,Fundamentals,Group by GVKey,All,"(348, 167)"


# Join all

* Fundamentals
* Stocks
* Securities
* Ratings
* SCA Filings

Result should have 348 rows, 189 columns

In [951]:
# Create copy for all
completeDF = fundamentals.copy()

# Clean Actions Report
cleanActionsDesc = "Make CompleteDF from a copy of fundaments"
df_name = "completeDF"
cols = "All fundamentals cols"

cleanActionsReport(completeDF,df_name,cleanActionsDesc,cols)


,DF,ActionDesc,List-of-Cols,ResultShape
0,Fundamentals,Dropped obvious low value columns -- 1 -- Date...,"[datadate, apdedate, fdate, pdate, fyr, add1, ...","(2323, 1744)"
1,Fundamentals,Initial Null Drop -- columns w/ 63% Null or more.,"[acctchg, acqmeth, adrr, bspr, compst, curuscn...","(2323, 326)"
2,Fundamentals,Dropped columns with only 1 unique value,"[indfmt, consol, popsrc, curcd, final, scf, ac...","(2323, 310)"
3,Fundamentals,Dropping all that are not standard statements ...,"applies to all cols, roughly 50% of rows dropped","(1243, 310)"
4,Fundamentals,Second Null Drop -- columns w/ 20% Null or more.,"[cld2, cld3, cld4, cld5, cshr, dd3, dd4, dd5, ...","(1243, 265)"
5,Fundamentals,Dropping Columns w/ Only 2 values,"[datafmt, ismod, upd, rdip, rdipa, rdipd, rdip...","(1243, 250)"
6,Fundamentals,Second Manual Column Reduce - Dropping Identif...,"[Unnamed: 0, conm, au, auopic, pddur, dpact_fn...","(1243, 240)"
7,Fundamentals,Third Manual Column Reduce - Targetted Reduce...,"[lco, lcox, lcoxdr, lo, loxdr, mib, mibn, ppev...","(1243, 92)"
8,Fundamentals,"Summary Features - Used to create features, t...","[drc, drlt, rea, reajo, recta]","(1243, 88)"
9,Fundamentals,Group by GVKey,All,"(348, 167)"


In [952]:
# Join Stocks
stocks = stocks.set_index('gvkey')
stocks = stocks.add_prefix(prefix="st_")
completeDF = completeDF.join(stocks)

# Clean Actions Report
cleanActionsDesc = "Add Stocks to CompleteDF"
df_name = "completeDF"
cols = "fundamentals + stocks"

cleanActionsReport(completeDF,df_name,cleanActionsDesc,cols)

,DF,ActionDesc,List-of-Cols,ResultShape
0,Fundamentals,Dropped obvious low value columns -- 1 -- Date...,"[datadate, apdedate, fdate, pdate, fyr, add1, ...","(2323, 1744)"
1,Fundamentals,Initial Null Drop -- columns w/ 63% Null or more.,"[acctchg, acqmeth, adrr, bspr, compst, curuscn...","(2323, 326)"
2,Fundamentals,Dropped columns with only 1 unique value,"[indfmt, consol, popsrc, curcd, final, scf, ac...","(2323, 310)"
3,Fundamentals,Dropping all that are not standard statements ...,"applies to all cols, roughly 50% of rows dropped","(1243, 310)"
4,Fundamentals,Second Null Drop -- columns w/ 20% Null or more.,"[cld2, cld3, cld4, cld5, cshr, dd3, dd4, dd5, ...","(1243, 265)"
5,Fundamentals,Dropping Columns w/ Only 2 values,"[datafmt, ismod, upd, rdip, rdipa, rdipd, rdip...","(1243, 250)"
6,Fundamentals,Second Manual Column Reduce - Dropping Identif...,"[Unnamed: 0, conm, au, auopic, pddur, dpact_fn...","(1243, 240)"
7,Fundamentals,Third Manual Column Reduce - Targetted Reduce...,"[lco, lcox, lcoxdr, lo, loxdr, mib, mibn, ppev...","(1243, 92)"
8,Fundamentals,"Summary Features - Used to create features, t...","[drc, drlt, rea, reajo, recta]","(1243, 88)"
9,Fundamentals,Group by GVKey,All,"(348, 167)"


In [953]:
# Join Securities
securities = securities.set_index('gvkey')
securities = securities.add_prefix(prefix="sec_")
completeDF = completeDF.join(securities)

# Clean Actions Report
cleanActionsDesc = "Add Securities to CompleteDF"
df_name = "completeDF"
cols = "fundamentals + stocks + securites"

cleanActionsReport(completeDF,df_name,cleanActionsDesc,cols)


,DF,ActionDesc,List-of-Cols,ResultShape
0,Fundamentals,Dropped obvious low value columns -- 1 -- Date...,"[datadate, apdedate, fdate, pdate, fyr, add1, ...","(2323, 1744)"
1,Fundamentals,Initial Null Drop -- columns w/ 63% Null or more.,"[acctchg, acqmeth, adrr, bspr, compst, curuscn...","(2323, 326)"
2,Fundamentals,Dropped columns with only 1 unique value,"[indfmt, consol, popsrc, curcd, final, scf, ac...","(2323, 310)"
3,Fundamentals,Dropping all that are not standard statements ...,"applies to all cols, roughly 50% of rows dropped","(1243, 310)"
4,Fundamentals,Second Null Drop -- columns w/ 20% Null or more.,"[cld2, cld3, cld4, cld5, cshr, dd3, dd4, dd5, ...","(1243, 265)"
5,Fundamentals,Dropping Columns w/ Only 2 values,"[datafmt, ismod, upd, rdip, rdipa, rdipd, rdip...","(1243, 250)"
6,Fundamentals,Second Manual Column Reduce - Dropping Identif...,"[Unnamed: 0, conm, au, auopic, pddur, dpact_fn...","(1243, 240)"
7,Fundamentals,Third Manual Column Reduce - Targetted Reduce...,"[lco, lcox, lcoxdr, lo, loxdr, mib, mibn, ppev...","(1243, 92)"
8,Fundamentals,"Summary Features - Used to create features, t...","[drc, drlt, rea, reajo, recta]","(1243, 88)"
9,Fundamentals,Group by GVKey,All,"(348, 167)"


In [954]:
# Join Ratings
ratings = ratings.set_index('gvkey')
ratings = ratings.add_prefix(prefix="rat_")
completeDF = completeDF.join(ratings)

# Clean Actions Report
cleanActionsDesc = "Add Ratings to CompleteDF"
df_name = "completeDF"
cols = "fundamentals + stocks + securites + ratings"

cleanActionsReport(completeDF,df_name,cleanActionsDesc,cols)

,DF,ActionDesc,List-of-Cols,ResultShape
0,Fundamentals,Dropped obvious low value columns -- 1 -- Date...,"[datadate, apdedate, fdate, pdate, fyr, add1, ...","(2323, 1744)"
1,Fundamentals,Initial Null Drop -- columns w/ 63% Null or more.,"[acctchg, acqmeth, adrr, bspr, compst, curuscn...","(2323, 326)"
2,Fundamentals,Dropped columns with only 1 unique value,"[indfmt, consol, popsrc, curcd, final, scf, ac...","(2323, 310)"
3,Fundamentals,Dropping all that are not standard statements ...,"applies to all cols, roughly 50% of rows dropped","(1243, 310)"
4,Fundamentals,Second Null Drop -- columns w/ 20% Null or more.,"[cld2, cld3, cld4, cld5, cshr, dd3, dd4, dd5, ...","(1243, 265)"
5,Fundamentals,Dropping Columns w/ Only 2 values,"[datafmt, ismod, upd, rdip, rdipa, rdipd, rdip...","(1243, 250)"
6,Fundamentals,Second Manual Column Reduce - Dropping Identif...,"[Unnamed: 0, conm, au, auopic, pddur, dpact_fn...","(1243, 240)"
7,Fundamentals,Third Manual Column Reduce - Targetted Reduce...,"[lco, lcox, lcoxdr, lo, loxdr, mib, mibn, ppev...","(1243, 92)"
8,Fundamentals,"Summary Features - Used to create features, t...","[drc, drlt, rea, reajo, recta]","(1243, 88)"
9,Fundamentals,Group by GVKey,All,"(348, 167)"


In [955]:
#reset index of completeDF before final join which uses tic
completeDF = completeDF.reset_index()

In [956]:
# Join SCA Filings
completeDF = completeDF.merge(sca_filings, left_on='tic', right_on='tic', how='left')

# Clean Actions Report
cleanActionsDesc = "Add SCA Filings to CompleteDF"
df_name = "completeDF"
cols = "fundamentals + stocks + securites + ratings + SCA"

cleanActionsReport(completeDF,df_name,cleanActionsDesc,cols)


,DF,ActionDesc,List-of-Cols,ResultShape
0,Fundamentals,Dropped obvious low value columns -- 1 -- Date...,"[datadate, apdedate, fdate, pdate, fyr, add1, ...","(2323, 1744)"
1,Fundamentals,Initial Null Drop -- columns w/ 63% Null or more.,"[acctchg, acqmeth, adrr, bspr, compst, curuscn...","(2323, 326)"
2,Fundamentals,Dropped columns with only 1 unique value,"[indfmt, consol, popsrc, curcd, final, scf, ac...","(2323, 310)"
3,Fundamentals,Dropping all that are not standard statements ...,"applies to all cols, roughly 50% of rows dropped","(1243, 310)"
4,Fundamentals,Second Null Drop -- columns w/ 20% Null or more.,"[cld2, cld3, cld4, cld5, cshr, dd3, dd4, dd5, ...","(1243, 265)"
5,Fundamentals,Dropping Columns w/ Only 2 values,"[datafmt, ismod, upd, rdip, rdipa, rdipd, rdip...","(1243, 250)"
6,Fundamentals,Second Manual Column Reduce - Dropping Identif...,"[Unnamed: 0, conm, au, auopic, pddur, dpact_fn...","(1243, 240)"
7,Fundamentals,Third Manual Column Reduce - Targetted Reduce...,"[lco, lcox, lcoxdr, lo, loxdr, mib, mibn, ppev...","(1243, 92)"
8,Fundamentals,"Summary Features - Used to create features, t...","[drc, drlt, rea, reajo, recta]","(1243, 88)"
9,Fundamentals,Group by GVKey,All,"(348, 167)"


# Exports

In [957]:
# Export Cleaned Data to CSV:
fundamentals.to_csv(cleanDataPath+'fundamental_cleaned.csv')
ratings.to_csv(cleanDataPath+'ratings_cleaned.csv')
securities.to_csv(cleanDataPath+'securities_cleaned.csv')
stocks.to_csv(cleanDataPath+'stocks_cleaned.csv')
sca_filings.to_csv(cleanDataPath+'sca_cleaned.csv')
completeDF.to_csv(cleanDataPath+'completeDF.csv')

# Export Clean Actions Report:
cleanActionsDF.to_excel(exportPath+"Clean_Actions_Report.xlsx")
print("Done")





Done
